<a href="https://colab.research.google.com/github/anaghavs09/RebalanceAI-AI-Powered-Investment-Portfolio-Analyzer/blob/main/Final_Hackathon_Group_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 RebalanceAI: Complete AI-Powered Investment Portfolio Analyzer

## Welcome to Your Intelligent Investment Co-Pilot

**RebalanceAI** leverages LangChain agents and Large Language Models to transform raw market data into actionable investment insights. This system combines:

- 📊 **Quantitative Finance**: Technical indicators, risk metrics, and fundamental analysis
- 💬 **AI Sentiment Analysis**: Real-time news and social media sentiment scoring
- 🤖 **LangChain Agents**: Intelligent tool orchestration for autonomous portfolio analysis
- 🎯 **Risk-Aware Optimization**: Personalized allocations based on your risk profile
- 📅 **Economic Calendar**: Event-aware position sizing and timing recommendations

### What This Notebook Does

1. **Data Collection**: Fetch S&P 500 stock data, fundamentals, and market sentiment
2. **Quantitative Analysis**: Calculate volatility, Sharpe ratios, correlations, and technical indicators
3. **Sentiment Fusion**: Integrate LLM-powered sentiment from news and social platforms
4. **Economic Calendar**: Analyze upcoming events impact on portfolio decisions
5. **AI Agent Orchestration**: Use LangChain to coordinate tools and generate recommendations
6. **Portfolio Optimization**: Produce explainable, risk-adjusted allocation strategies


Let's begin by setting up our environment! 🎯

## 🔧 Environment Setup & Installation

### Install Required Dependencies

Installing all necessary libraries for financial data analysis, AI agent orchestration, and portfolio optimization. This includes:
- **Financial Data**: yfinance for market data
- **Data Processing**: pandas, numpy for analysis
- **Visualization**: matplotlib, plotly for charts
- **AI Framework**: LangChain for agent architecture
- **LLM Integration**: OpenAI, Google Gemini
- **UI**: Gradio for interactive interface
- **ML Tools**: scikit-learn for optimization
- **Sentiment Analysis**: PRAW for Reddit data

In [1]:
!pip install yfinance pandas numpy matplotlib plotly langchain langchain-openai gradio scikit-learn praw -q
print("\n📦 Installing additional LLM providers...")
!pip install -q google-generativeai langchain-google-genai
print("✅ Installation complete!")


📦 Installing additional LLM providers...
✅ Installation complete!


### Import Core Libraries & Configure API Keys

Importing essential modules for the RebalanceAI system:
- **Financial Analysis**: yfinance, pandas, numpy for data processing
- **LangChain Framework**: ReAct agent, tools, memory, and prompts for agent orchestration
- **LLM Integration**: OpenAI ChatGPT for intelligent reasoning and decision-making
- **Visualization**: Plotly for interactive portfolio charts
- **UI**: Gradio for user-friendly interface

**ReAct Agent Architecture**: The agent uses Reasoning and Acting (ReAct) pattern to iteratively think through investment problems, select appropriate analytical tools, and execute actions based on market observations.

In [3]:
# ===================================================================
# RebalanceAI - Complete Environment Setup & Configuration
# ===================================================================

import os
import warnings
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
import json

warnings.filterwarnings('ignore')

# Core data science libraries
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import praw

# LangChain imports for agent orchestration
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# UI and visualization
import gradio as gr
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ===================================================================
# Configuration & Global Settings
# ===================================================================

# Top 20 S&P 500 stocks by market cap (diverse sectors)
TICKERS = ['AAPL','MSFT','GOOGL','AMZN','NVDA','TSLA','META','BRK-B','UNH','JNJ',
           'V','WMT','XOM','LLY','JPM','PG','MA','HD','CVX','ABBV']

TICKER_NAMES = {
    "AAPL": "Apple", "MSFT": "Microsoft", "GOOGL": "Google", "AMZN": "Amazon",
    "NVDA": "Nvidia", "TSLA": "Tesla", "META": "Meta", "BRK-B": "Berkshire",
    "UNH": "UnitedHealth", "JNJ": "Johnson & Johnson", "V": "Visa", "WMT": "Walmart",
    "XOM": "ExxonMobil", "LLY": "Eli Lilly", "JPM": "JPMorgan", "PG": "Procter & Gamble",
    "MA": "Mastercard", "HD": "Home Depot", "CVX": "Chevron", "ABBV": "AbbVie"
}

# Sector mapping for economic analysis
STOCK_SECTORS = {
    'AAPL': 'Technology', 'MSFT': 'Technology', 'GOOGL': 'Technology',
    'NVDA': 'Technology', 'META': 'Technology', 'TSLA': 'Consumer Discretionary',
    'JPM': 'Financials', 'V': 'Financials', 'MA': 'Financials', 'BRK-B': 'Financials',
    'JNJ': 'Healthcare', 'UNH': 'Healthcare', 'LLY': 'Healthcare',
    'WMT': 'Consumer Staples', 'PG': 'Consumer Staples',
    'XOM': 'Energy', 'CVX': 'Energy', 'HD': 'Consumer Discretionary', 'ABBV': 'Healthcare',
    'AMZN': 'Consumer Discretionary'
}

# Load API keys from Colab secrets
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
NEWSAPI_KEY = userdata.get('NEWSAPI_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
REDDIT_CLIENT_ID = userdata.get('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET = userdata.get('REDDIT_CLIENT_SECRET')
REDDIT_USER_AGENT = userdata.get('REDDIT_USER_AGENT')

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Initialize Reddit client
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# Global cache for sharing data between tools
CACHE = {}

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

print("✅ RebalanceAI Configuration Complete!")
print(f"📊 Stock Universe: {len(TICKERS)} tickers")
print(f"🔑 APIs Configured: OpenAI, NewsAPI, Reddit, Google")
print(f"🤖 LLM Model: GPT-4o-mini")

✅ RebalanceAI Configuration Complete!
📊 Stock Universe: 20 tickers
🔑 APIs Configured: OpenAI, NewsAPI, Reddit, Google
🤖 LLM Model: GPT-4o-mini


## ⚙️ Helper Functions & Technical Analysis

### Core Quantitative Finance Functions

Implementing essential technical analysis calculations:
- **RSI (Relative Strength Index)**: Momentum oscillator (0-100 scale)
- **Technical Score**: Composite metric combining RSI, MACD, and Bollinger Bands
- **Fundamental Score**: Multi-factor analysis of P/E, growth, margins, and ROE
- **Input Parsing**: Handle various LLM parameter formats

These functions form the quantitative backbone of our analysis system.

In [4]:
# ===================================================================
# Helper Functions & Technical Analysis
# ===================================================================

def calculate_rsi(prices, period=14):
    """
    Calculate Relative Strength Index (RSI) with proper error handling
    RSI ranges from 0-100: <30 oversold, >70 overbought
    """
    try:
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi.fillna(50)  # Neutral value for NaN
    except:
        return pd.Series([50] * len(prices), index=prices.index)

def calculate_technical_score(rsi, macd_signal, bb_position):
    """
    Calculate comprehensive technical strength score (-3 to +3 range)
    Combines multiple technical indicators for overall signal strength
    """
    score = 0

    # RSI component: oversold/overbought signals
    if rsi < 30:
        score += 1  # Oversold = potential buy signal
    elif rsi > 70:
        score -= 1  # Overbought = potential sell signal

    # MACD component: trend direction
    score += macd_signal * 0.5

    # Bollinger Band position: volatility-based signals
    if bb_position < 0.2:
        score += 0.5  # Near lower band = potential buy
    elif bb_position > 0.8:
        score -= 0.5  # Near upper band = potential sell

    return score

def calculate_fundamental_score(pe, growth, margin, roe):
    """
    Calculate fundamental strength score (0-4 range)
    Evaluates company's financial health and valuation attractiveness
    """
    score = 0

    # P/E ratio evaluation (valuation metric)
    if 0 < pe < 15:
        score += 1  # Attractive valuation
    elif pe > 30:
        score -= 0.5  # Potentially overvalued

    # Revenue growth evaluation
    if growth > 0.15:
        score += 1  # Strong growth (>15%)
    elif growth < 0:
        score -= 0.5  # Declining revenue

    # Profit margin evaluation
    if margin > 0.20:
        score += 0.5  # Healthy margins (>20%)

    # Return on Equity evaluation
    if roe > 0.15:
        score += 0.5  # Efficient use of equity (>15%)

    return score

def parse_tickers_input(input_string):
    """
    Parse ticker input handling various LLM output formats
    Handles: "AAPL,MSFT", "tickers=AAPL,MSFT", "AAPL MSFT", etc.
    """
    if not input_string:
        return []

    # Remove common LLM parameter prefixes
    input_clean = input_string.replace('tickers=', '').replace('ticker=', '').strip()

    # Handle different delimiters
    if ',' in input_clean:
        tickers = [t.strip().upper() for t in input_clean.split(',')]
    else:
        tickers = [t.strip().upper() for t in input_clean.split()]

    # Filter to valid tickers and limit for performance
    valid_tickers = [t for t in tickers if t in TICKERS]
    return valid_tickers[:10]  # Limit to 10 for performance

def get_stock_sector(ticker: str) -> str:
    """Get stock sector for economic impact analysis"""
    return STOCK_SECTORS.get(ticker, 'Mixed')

print("✅ Helper functions loaded!")
print(f"📊 Technical Analysis: RSI, MACD, Bollinger Bands")
print(f"💼 Fundamental Analysis: P/E, Growth, Margins, ROE")
print(f"🔧 Utility Functions: Input parsing, sector mapping")

✅ Helper functions loaded!
📊 Technical Analysis: RSI, MACD, Bollinger Bands
💼 Fundamental Analysis: P/E, Growth, Margins, ROE
🔧 Utility Functions: Input parsing, sector mapping


## 🛠️ Tool 1 - Market Data Gatherer

### Comprehensive Market Analysis Engine

This tool fetches and processes:
- **Price & Performance**: Current price, returns, volatility, Sharpe ratio
- **Technical Indicators**: RSI, MACD, Bollinger Bands, moving averages
- **Fundamental Metrics**: P/E ratios, growth rates, profit margins, ROE
- **Composite Scoring**: Technical and fundamental strength scores

**Data Sources**: Yahoo Finance for comprehensive market data
**Caching**: Intelligent caching to optimize API usage and response times

In [5]:
# ===================================================================
# Tool 1: Market Data Gatherer
# ===================================================================

def gather_market_data_tool(tickers: str) -> str:
    """
    Gather comprehensive technical and fundamental metrics with full analysis.
    Returns detailed market data with composite scoring for investment decisions.
    """
    ticker_list = parse_tickers_input(tickers)
    results = []

    for ticker in ticker_list:
        try:
            print(f"📊 Processing {ticker} ({TICKER_NAMES.get(ticker, ticker)})...")

            # Fetch data from Yahoo Finance
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1y")
            info = stock.info

            if hist.empty:
                results.append(f"{ticker}: ❌ No data available")
                continue

            # === TECHNICAL ANALYSIS SECTION ===
            returns = hist['Close'].pct_change().dropna()
            current_price = hist['Close'].iloc[-1]

            # Moving averages (multiple timeframes)
            hist['SMA_20'] = hist['Close'].rolling(20).mean()
            hist['SMA_50'] = hist['Close'].rolling(50).mean()
            hist['SMA_200'] = hist['Close'].rolling(200).mean()

            # RSI calculation
            hist['RSI'] = calculate_rsi(hist['Close'])

            # MACD calculation (trend indicator)
            exp1 = hist['Close'].ewm(span=12, adjust=False).mean()
            exp2 = hist['Close'].ewm(span=26, adjust=False).mean()
            hist['MACD'] = exp1 - exp2
            hist['Signal'] = hist['MACD'].ewm(span=9, adjust=False).mean()

            # Bollinger Bands (volatility indicator)
            sma20 = hist['Close'].rolling(20).mean()
            std20 = hist['Close'].rolling(20).std()
            hist['BB_upper'] = sma20 + (std20 * 2)
            hist['BB_lower'] = sma20 - (std20 * 2)

            # Volume analysis
            hist['Volume_SMA'] = hist['Volume'].rolling(20).mean()
            volume_ratio = (hist['Volume'].iloc[-1] / hist['Volume_SMA'].iloc[-1]
                          if hist['Volume_SMA'].iloc[-1] > 0 else 1)

            # === FUNDAMENTAL ANALYSIS SECTION ===
            pe_ratio = info.get('forwardPE', info.get('trailingPE', 0))
            pb_ratio = info.get('priceToBook', 0)
            revenue_growth = info.get('revenueGrowth', 0)
            earnings_growth = info.get('earningsGrowth', 0)
            profit_margin = info.get('profitMargins', 0)
            roe = info.get('returnOnEquity', 0)
            debt_equity = info.get('debtToEquity', 0) / 100 if info.get('debtToEquity') else 0
            current_ratio = info.get('currentRatio', 0)
            beta = info.get('beta', 1.0)
            market_cap = info.get('marketCap', 0)

            # === PERFORMANCE CALCULATIONS ===
            annual_return = returns.mean() * 252 if len(returns) > 0 else 0
            volatility = returns.std() * np.sqrt(252) if len(returns) > 0 else 0
            sharpe = (annual_return - 0.02) / volatility if volatility > 0 else 0

            # Technical signal calculations
            price_vs_sma20 = ((current_price - hist['SMA_20'].iloc[-1]) / hist['SMA_20'].iloc[-1]
                             if hist['SMA_20'].iloc[-1] > 0 else 0)
            macd_signal = 1 if hist['MACD'].iloc[-1] > hist['Signal'].iloc[-1] else -1
            bb_position = ((current_price - hist['BB_lower'].iloc[-1]) /
                          (hist['BB_upper'].iloc[-1] - hist['BB_lower'].iloc[-1])
                          if (hist['BB_upper'].iloc[-1] - hist['BB_lower'].iloc[-1]) > 0 else 0.5)

            # === COMPOSITE SCORING ===
            technical_score = calculate_technical_score(
                hist['RSI'].iloc[-1], macd_signal, bb_position
            )

            fundamental_score = calculate_fundamental_score(
                pe_ratio, revenue_growth, profit_margin, roe
            )

            # Store comprehensive data in cache
            if 'market_data' not in CACHE:
                CACHE['market_data'] = {}

            CACHE['market_data'][ticker] = {
                # Price & Performance
                'current_price': current_price,
                'annual_return': annual_return,
                'volatility': volatility,
                'sharpe': sharpe,
                'beta': beta,

                # Technical Indicators
                'rsi': hist['RSI'].iloc[-1],
                'macd_signal': macd_signal,
                'price_vs_sma20': price_vs_sma20,
                'bb_position': bb_position,
                'volume_ratio': volume_ratio,

                # Moving Averages
                'sma_20': hist['SMA_20'].iloc[-1],
                'sma_50': hist['SMA_50'].iloc[-1],
                'sma_200': hist['SMA_200'].iloc[-1] if len(hist) > 200 else hist['SMA_50'].iloc[-1],

                # Fundamentals
                'pe_ratio': pe_ratio,
                'pb_ratio': pb_ratio,
                'revenue_growth': revenue_growth,
                'earnings_growth': earnings_growth,
                'profit_margin': profit_margin,
                'roe': roe,
                'debt_equity': debt_equity,
                'current_ratio': current_ratio,
                'market_cap': market_cap,

                # Composite Scores
                'technical_score': technical_score,
                'fundamental_score': fundamental_score,
                'timestamp': time.time()
            }

            results.append(
                f"{ticker}: Price=${current_price:.2f}, Sharpe={sharpe:.2f}, "
                f"PE={pe_ratio:.1f}, Growth={revenue_growth:.1%}, RSI={hist['RSI'].iloc[-1]:.1f}"
            )

        except Exception as e:
            results.append(f"{ticker}: ❌ Error - {str(e)}")
            print(f"Error processing {ticker}: {e}")

    return "Market Data Analysis:\n" + "\n".join(results)

# Create LangChain Tool
market_data_tool = Tool(
    name="gather_market_data",
    func=gather_market_data_tool,
    description="Gather comprehensive market data including technical indicators, fundamentals, and performance metrics. Input: ticker symbols like AAPL or AAPL,MSFT,GOOGL"
)

print("✅ Market Data Tool ready!")

✅ Market Data Tool ready!


## 📊 DEMO: Market Data Analysis Tool

Let's test our market data tool with real stocks and see the comprehensive analysis it provides. This demo will show:

1. **Raw tool output** with summary metrics
2. **Detailed cached data** showing all calculated indicators  
3. **Technical analysis** breakdown with signal interpretation
4. **Fundamental analysis** with valuation metrics
5. **Composite scoring** system in action

**Test Stocks**: NVDA (Nvidia), TSLA (Tesla), AAPL (Apple) - representing different market dynamics

In [6]:
# ===================================================================
# DEMO: Market Data Gathering Tool
# ===================================================================

print("="*70)
print("🔍 DEMO: COMPREHENSIVE MARKET DATA ANALYSIS")
print("="*70)

# Test with high-profile stocks
test_tickers = "NVDA,TSLA,AAPL"
print(f"\n📊 Testing comprehensive analysis for: {test_tickers}")
print("-"*50)

# Execute the tool
print("\n1️⃣ RAW TOOL OUTPUT:")
print("-"*50)
market_result = gather_market_data_tool(test_tickers)
print(market_result)

# Show detailed breakdown of cached data
print("\n2️⃣ DETAILED ANALYSIS BREAKDOWN:")
print("-"*50)

for ticker in test_tickers.split(','):
    if ticker.strip() in CACHE.get('market_data', {}):
        print(f"\n{'='*60}")
        print(f"📈 {ticker} - {TICKER_NAMES.get(ticker, ticker)} Deep Dive")
        print(f"{'='*60}")

        data = CACHE['market_data'][ticker.strip()]

        # Price & Performance Section
        print(f"\n💰 PRICE & PERFORMANCE:")
        print(f"  • Current Price: ${data['current_price']:.2f}")
        print(f"  • Annual Return: {data['annual_return']:.2%}")
        print(f"  • Volatility: {data['volatility']:.2%}")
        print(f"  • Sharpe Ratio: {data['sharpe']:.3f}", end="")
        if data['sharpe'] > 1:
            print(" (🟢 Excellent)")
        elif data['sharpe'] > 0.5:
            print(" (🟡 Good)")
        else:
            print(" (🔴 Below Average)")
        print(f"  • Beta: {data['beta']:.2f} (Market sensitivity)")

        # Technical Analysis Section
        print(f"\n📉 TECHNICAL INDICATORS:")
        rsi_val = data['rsi']
        print(f"  • RSI: {rsi_val:.2f}", end=" ")
        if rsi_val > 70:
            print("(⚠️ Overbought - Potential sell signal)")
        elif rsi_val < 30:
            print("(✅ Oversold - Potential buy signal)")
        else:
            print("(➖ Neutral zone)")

        print(f"  • MACD Signal: {'🟢 Bullish' if data['macd_signal'] > 0 else '🔴 Bearish'}")
        print(f"  • Price vs 20-day SMA: {data['price_vs_sma20']:.2%}")

        bb_pos = data['bb_position']
        print(f"  • Bollinger Band Position: {bb_pos:.1%}", end=" ")
        if bb_pos < 0.2:
            print("(Near lower band - oversold)")
        elif bb_pos > 0.8:
            print("(Near upper band - overbought)")
        else:
            print("(Middle range)")

        print(f"  • Volume Ratio: {data['volume_ratio']:.2f}x average")

        # Fundamental Analysis Section
        print(f"\n💼 FUNDAMENTAL METRICS:")
        print(f"  • P/E Ratio: {data['pe_ratio']:.2f}", end=" ")
        if 0 < data['pe_ratio'] < 15:
            print("(Attractive valuation)")
        elif data['pe_ratio'] > 30:
            print("(Potentially overvalued)")
        else:
            print("(Reasonable valuation)")

        print(f"  • P/B Ratio: {data['pb_ratio']:.2f}")
        print(f"  • Revenue Growth: {data['revenue_growth']:.2%}")
        print(f"  • Profit Margin: {data['profit_margin']:.2%}")
        print(f"  • Return on Equity: {data['roe']:.2%}")
        print(f"  • Debt/Equity: {data['debt_equity']:.2f}")
        print(f"  • Market Cap: ${data['market_cap']:,.0f}" if data['market_cap'] > 0 else "  • Market Cap: N/A")

        # Composite Scoring Section
        print(f"\n🎯 COMPOSITE ANALYSIS:")
        tech_score = data['technical_score']
        fund_score = data['fundamental_score']

        print(f"  • Technical Score: {tech_score:.2f}/3.0", end=" ")
        if tech_score > 1:
            print("(🟢 Strong technical setup)")
        elif tech_score > 0:
            print("(🟡 Neutral technical)")
        else:
            print("(🔴 Weak technical setup)")

        print(f"  • Fundamental Score: {fund_score:.2f}/4.0", end=" ")
        if fund_score > 2:
            print("(🟢 Strong fundamentals)")
        elif fund_score > 1:
            print("(🟡 Decent fundamentals)")
        else:
            print("(🔴 Weak fundamentals)")

        # Overall Assessment
        print(f"\n📋 INVESTMENT ASSESSMENT:")
        overall_appeal = tech_score + fund_score
        if overall_appeal > 3:
            assessment = "🟢 STRONG BUY candidate"
        elif overall_appeal > 1:
            assessment = "🟡 HOLD/MODERATE interest"
        elif overall_appeal > -1:
            assessment = "🔴 CAUTION advised"
        else:
            assessment = "🔴 AVOID recommendation"

        print(f"  • Overall Appeal: {assessment}")

        # Risk Assessment
        print(f"  • Risk Profile:", end=" ")
        if data['volatility'] > 0.4:
            print("🔴 High volatility")
        elif data['volatility'] > 0.25:
            print("🟡 Moderate volatility")
        else:
            print("🟢 Low volatility")

        print(f"  • Sector: {STOCK_SECTORS.get(ticker.strip(), 'Unknown')}")

print(f"\n{'='*70}")
print("✅ Market Data Tool Demo Complete!")
print(f"📊 Cached data for {len(CACHE.get('market_data', {}))} stocks")
print(f"🎯 Ready for sentiment analysis and portfolio allocation")
print("="*70)

🔍 DEMO: COMPREHENSIVE MARKET DATA ANALYSIS

📊 Testing comprehensive analysis for: NVDA,TSLA,AAPL
--------------------------------------------------

1️⃣ RAW TOOL OUTPUT:
--------------------------------------------------
📊 Processing NVDA (Nvidia)...
📊 Processing TSLA (Tesla)...
📊 Processing AAPL (Apple)...
Market Data Analysis:
NVDA: Price=$183.22, Sharpe=0.79, PE=44.5, Growth=55.6%, RSI=51.5
TSLA: Price=$439.31, Sharpe=1.30, PE=135.6, Growth=-11.8%, RSI=48.9
AAPL: Price=$252.29, Sharpe=0.33, PE=30.4, Growth=9.6%, RSI=46.5

2️⃣ DETAILED ANALYSIS BREAKDOWN:
--------------------------------------------------

📈 NVDA - Nvidia Deep Dive

💰 PRICE & PERFORMANCE:
  • Current Price: $183.22
  • Annual Return: 41.03%
  • Volatility: 49.58%
  • Sharpe Ratio: 0.787 (🟡 Good)
  • Beta: 2.12 (Market sensitivity)

📉 TECHNICAL INDICATORS:
  • RSI: 51.53 (➖ Neutral zone)
  • MACD Signal: 🔴 Bearish
  • Price vs 20-day SMA: -0.31%
  • Bollinger Band Position: 46.8% (Middle range)
  • Volume Ratio: 0.95x

## 💭 Tool 2 - Sentiment Analysis Engine

### Multi-Source Sentiment Intelligence

This tool combines sentiment from multiple sources:
- **NewsAPI Integration**: Real-time financial news sentiment scoring
- **Reddit Analysis**: Social media sentiment from investing communities  
- **Intelligent Weighting**: 70% news + 30% Reddit for balanced perspective
- **Keyword-Based Scoring**: Advanced sentiment analysis using financial terms

**Data Sources**:
- NewsAPI for professional financial news
- Reddit (r/wallstreetbets, r/stocks, r/investing) for retail sentiment
- 7-day rolling window for current market mood

In [7]:
# ===================================================================
# Tool 2: Sentiment Analysis Engine
# ===================================================================

def get_news_sentiment(ticker: str) -> Dict[str, Any]:
    """
    Get comprehensive news sentiment from NewsAPI with financial keyword analysis
    """
    try:
        company_name = TICKER_NAMES.get(ticker, ticker)

        # NewsAPI configuration
        url = 'https://newsapi.org/v2/everything'
        to_date = datetime.now()
        from_date = to_date - timedelta(days=7)

        params = {
            'q': f'"{company_name}" OR "{ticker}"',
            'from': from_date.strftime('%Y-%m-%d'),
            'to': to_date.strftime('%Y-%m-%d'),
            'sortBy': 'relevancy',
            'language': 'en',
            'apiKey': NEWSAPI_KEY,
            'pageSize': 10
        }

        response = requests.get(url, params=params, timeout=30)

        if response.status_code != 200:
            print(f"NewsAPI error for {ticker}: {response.status_code}")
            return {'score': 0, 'confidence': 0.1, 'article_count': 0}

        data = response.json()
        articles = data.get('articles', [])

        if not articles:
            return {'score': 0, 'confidence': 0.1, 'article_count': 0}

        # Financial sentiment keywords (expanded)
        positive_words = [
            'surge', 'gain', 'rise', 'profit', 'growth', 'positive', 'strong', 'beat',
            'exceed', 'breakthrough', 'innovative', 'bullish', 'upgrade', 'outperform',
            'record', 'momentum', 'rally', 'boost', 'soar', 'climb'
        ]

        negative_words = [
            'fall', 'drop', 'loss', 'decline', 'concern', 'risk', 'weak', 'miss',
            'below', 'problem', 'issue', 'bearish', 'downgrade', 'underperform',
            'crash', 'plunge', 'slump', 'warning', 'cut', 'reduce'
        ]

        sentiment_scores = []
        processed_headlines = []

        for article in articles:
            title = (article.get('title', '') + ' ' + article.get('description', '')).lower()

            # Count sentiment indicators
            pos_count = sum(1 for word in positive_words if word in title)
            neg_count = sum(1 for word in negative_words if word in title)

            # Calculate article sentiment
            if pos_count + neg_count > 0:
                article_sentiment = (pos_count - neg_count) / (pos_count + neg_count)
            else:
                article_sentiment = 0

            sentiment_scores.append(article_sentiment)
            processed_headlines.append({
                'title': article.get('title', ''),
                'sentiment': article_sentiment,
                'positive_words': pos_count,
                'negative_words': neg_count
            })

        # Calculate overall sentiment
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0

        # Confidence based on article count and sentiment consistency
        confidence = min(len(articles) / 10, 1.0)
        if sentiment_scores:
            std_dev = np.std(sentiment_scores)
            confidence *= max(0.3, 1 - std_dev)

        return {
            'score': avg_sentiment,
            'confidence': confidence,
            'article_count': len(articles),
            'headlines': processed_headlines
        }

    except Exception as e:
        print(f"News sentiment error for {ticker}: {e}")
        return {'score': 0, 'confidence': 0.1, 'article_count': 0, 'headlines': []}

def get_reddit_sentiment(ticker: str) -> Dict[str, Any]:
    """
    Gather Reddit sentiment from multiple investing subreddits
    """
    try:
        subreddits = ['wallstreetbets', 'stocks', 'investing']
        posts_data = []

        print(f"🔍 Searching Reddit for ${ticker} mentions...")

        for sub_name in subreddits:
            try:
                subreddit = reddit.subreddit(sub_name)

                # Search for ticker mentions in past week
                for post in subreddit.search(f"${ticker}", time_filter='week', limit=5):
                    posts_data.append({
                        'title': post.title,
                        'score': post.score,
                        'num_comments': post.num_comments,
                        'upvote_ratio': post.upvote_ratio,
                        'subreddit': sub_name,
                        'created': post.created_utc
                    })
            except Exception as e:
                print(f"Error accessing r/{sub_name}: {e}")
                continue

        if not posts_data:
            return {'score': 0, 'confidence': 0.1, 'volume': 0, 'posts': []}

        # Calculate Reddit sentiment metrics
        total_score = sum(p['score'] for p in posts_data)
        avg_upvote_ratio = sum(p['upvote_ratio'] for p in posts_data) / len(posts_data)
        mention_volume = len(posts_data)

        # Convert upvote ratio to sentiment (-1 to +1)
        sentiment_score = (avg_upvote_ratio - 0.5) * 2

        # Confidence based on volume and engagement
        confidence = min(mention_volume / 10, 1.0)
        avg_engagement = sum(p['num_comments'] for p in posts_data) / len(posts_data)
        confidence *= min(avg_engagement / 20, 1.0)  # Higher engagement = higher confidence

        return {
            'score': sentiment_score,
            'confidence': confidence,
            'volume': mention_volume,
            'total_karma': total_score,
            'avg_engagement': avg_engagement,
            'posts': [{'title': p['title'], 'score': p['score'], 'subreddit': p['subreddit']}
                     for p in posts_data[:3]]  # Top 3 posts for reference
        }

    except Exception as e:
        print(f"Reddit sentiment error for {ticker}: {e}")
        return {'score': 0, 'confidence': 0.1, 'volume': 0, 'posts': []}

def combined_sentiment_tool(tickers: str) -> str:
    """
    Sentiment analysis combining news and Reddit with intelligent weighting
    """
    ticker_list = parse_tickers_input(tickers)
    results = []

    for ticker in ticker_list:
        try:
            print(f"💭 Analyzing sentiment for {ticker} ({TICKER_NAMES.get(ticker, ticker)})...")

            # Get sentiment from both sources
            news_sentiment = get_news_sentiment(ticker)
            reddit_sentiment = get_reddit_sentiment(ticker)

            # Intelligent combination: 70% news (professional), 30% Reddit (retail)
            news_weight = 0.7
            reddit_weight = 0.3

            combined_score = (news_weight * news_sentiment['score'] +
                            reddit_weight * reddit_sentiment['score'])

            # Combined confidence (average of both sources)
            combined_confidence = (news_sentiment['confidence'] + reddit_sentiment['confidence']) / 2

            # Store comprehensive sentiment data
            if 'sentiment' not in CACHE:
                CACHE['sentiment'] = {}

            CACHE['sentiment'][ticker] = {
                'score': combined_score,
                'confidence': combined_confidence,
                'news_sentiment': news_sentiment['score'],
                'reddit_sentiment': reddit_sentiment['score'],
                'news_confidence': news_sentiment['confidence'],
                'reddit_confidence': reddit_sentiment['confidence'],
                'news_articles': news_sentiment['article_count'],
                'reddit_volume': reddit_sentiment['volume'],
                'reddit_karma': reddit_sentiment.get('total_karma', 0),
                'data_sources': {
                    'news': news_sentiment,
                    'reddit': reddit_sentiment
                },
                'timestamp': time.time()
            }

            # Generate result with emoji indicators
            if combined_score > 0.3:
                emoji = "🟢"
                interpretation = "Bullish"
            elif combined_score < -0.3:
                emoji = "🔴"
                interpretation = "Bearish"
            else:
                emoji = "🟡"
                interpretation = "Neutral"

            confidence_indicator = "🔥" if combined_confidence > 0.7 else "⚡" if combined_confidence > 0.4 else "💭"

            results.append(
                f"{ticker}: {emoji} {interpretation} {confidence_indicator} | "
                f"Combined={combined_score:+.2f} (News={news_sentiment['score']:+.2f}, "
                f"Reddit={reddit_sentiment['score']:+.2f}) | "
                f"Sources: {news_sentiment['article_count']} articles, {reddit_sentiment['volume']} posts"
            )

        except Exception as e:
            results.append(f"{ticker}: ❌ Sentiment analysis error - {str(e)}")

    return "Multi-Source Sentiment Analysis:\n" + "\n".join(results)

# Create LangChain Tool
sentiment_tool = Tool(
    name="analyze_sentiment",
    func=combined_sentiment_tool,
    description="Analyze market sentiment combining news and Reddit sources. Input: ticker symbols like AAPL or AAPL,MSFT,GOOGL"
)

print("✅ Sentiment Analysis Tool ready!")
print("📰 NewsAPI integration configured")
print("📱 Reddit API (WSB, stocks, investing) configured")

✅ Sentiment Analysis Tool ready!
📰 NewsAPI integration configured
📱 Reddit API (WSB, stocks, investing) configured


## 💭 DEMO: Sentiment Analysis

This demonstration shows our multi-source sentiment analysis in action. You'll see:

1. **Real-time news analysis** from financial media sources
2. **Reddit sentiment** from investing communities  
3. **Actual headlines and posts** that drive the sentiment scores
4. **Confidence scoring** based on data volume and consistency
5. **Combined intelligent weighting** (70% news + 30% Reddit)

**Test Stocks**: TSLA and NVDA - typically high social media engagement stocks

In [8]:
# ===================================================================
# DEMO: Sentiment Analysis with Real Data
# ===================================================================

print("="*70)
print("💭 DEMO: MULTI-SOURCE SENTIMENT ANALYSIS")
print("="*70)

test_sentiment_tickers = "TSLA,NVDA"
print(f"\n🎯 Analyzing sentiment for: {test_sentiment_tickers}")
print("These stocks typically have high social engagement and news coverage")
print("-"*50)

# Run sentiment analysis
print("\n📊 RUNNING COMPREHENSIVE SENTIMENT ANALYSIS...")
sentiment_result = combined_sentiment_tool(test_sentiment_tickers)
print("\n" + "="*50)
print("SENTIMENT ANALYSIS RESULTS:")
print("="*50)
print(sentiment_result)

# Detailed breakdown of sentiment analysis
print(f"\n{'='*70}")
print("🔍 DETAILED SENTIMENT BREAKDOWN")
print("="*70)

for ticker in test_sentiment_tickers.split(','):
    ticker = ticker.strip()
    if ticker in CACHE.get('sentiment', {}):
        print(f"\n{'='*60}")
        print(f"💭 {ticker} - {TICKER_NAMES.get(ticker, ticker)} Sentiment Deep Dive")
        print(f"{'='*60}")

        sentiment_data = CACHE['sentiment'][ticker]
        news_data = sentiment_data['data_sources']['news']
        reddit_data = sentiment_data['data_sources']['reddit']

        # Overall Sentiment Summary
        print(f"\n🎯 OVERALL SENTIMENT SUMMARY:")
        combined = sentiment_data['score']
        print(f"  • Combined Score: {combined:+.3f}", end=" ")
        if combined > 0.3:
            print("(🟢 Strong Bullish Sentiment)")
        elif combined > 0.1:
            print("(🟡 Mild Bullish Sentiment)")
        elif combined > -0.1:
            print("(🟡 Neutral Sentiment)")
        elif combined > -0.3:
            print("(🟡 Mild Bearish Sentiment)")
        else:
            print("(🔴 Strong Bearish Sentiment)")

        print(f"  • Confidence Level: {sentiment_data['confidence']:.1%}")
        print(f"  • Data Quality: {'🔥 High' if sentiment_data['confidence'] > 0.7 else '⚡ Medium' if sentiment_data['confidence'] > 0.4 else '💭 Low'}")

        # News Sentiment Analysis
        print(f"\n📰 NEWS SENTIMENT ANALYSIS:")
        print(f"  • News Score: {sentiment_data['news_sentiment']:+.3f}")
        print(f"  • Articles Analyzed: {sentiment_data['news_articles']}")
        print(f"  • News Confidence: {sentiment_data['news_confidence']:.1%}")

        if news_data.get('headlines'):
            print(f"  • Sample Headlines:")
            for i, headline in enumerate(news_data['headlines'][:3], 1):
                sentiment_indicator = "📈" if headline['sentiment'] > 0.2 else "📉" if headline['sentiment'] < -0.2 else "➡️"
                print(f"    {i}. {sentiment_indicator} {headline['title'][:80]}...")
                if headline['positive_words'] > 0 or headline['negative_words'] > 0:
                    print(f"       Signals: +{headline['positive_words']} positive, -{headline['negative_words']} negative")

        # Reddit Sentiment Analysis
        print(f"\n📱 REDDIT SENTIMENT ANALYSIS:")
        print(f"  • Reddit Score: {sentiment_data['reddit_sentiment']:+.3f}")
        print(f"  • Posts Found: {sentiment_data['reddit_volume']}")
        print(f"  • Total Karma: {sentiment_data['reddit_karma']:,}")
        print(f"  • Reddit Confidence: {sentiment_data['reddit_confidence']:.1%}")

        if reddit_data.get('posts'):
            print(f"  • Popular Reddit Posts:")
            for i, post in enumerate(reddit_data['posts'], 1):
                karma_indicator = "🔥" if post['score'] > 100 else "⚡" if post['score'] > 10 else "💭"
                print(f"    {i}. {karma_indicator} r/{post['subreddit']}: {post['title'][:60]}...")
                print(f"       Karma: {post['score']} points")

        # Sentiment Composition Analysis
        print(f"\n🧮 SENTIMENT COMPOSITION:")
        news_contribution = sentiment_data['news_sentiment'] * 0.7
        reddit_contribution = sentiment_data['reddit_sentiment'] * 0.3

        print(f"  • News Contribution (70%): {news_contribution:+.3f}")
        print(f"  • Reddit Contribution (30%): {reddit_contribution:+.3f}")
        print(f"  • Final Weighted Score: {news_contribution + reddit_contribution:+.3f}")

        # Data Quality Assessment
        print(f"\n📊 DATA QUALITY ASSESSMENT:")
        if sentiment_data['news_articles'] >= 5 and sentiment_data['reddit_volume'] >= 3:
            quality = "🟢 High Quality"
            reliability = "Very reliable sentiment reading"
        elif sentiment_data['news_articles'] >= 3 or sentiment_data['reddit_volume'] >= 2:
            quality = "🟡 Medium Quality"
            reliability = "Moderately reliable sentiment reading"
        else:
            quality = "🔴 Low Quality"
            reliability = "Limited data - sentiment may not be representative"

        print(f"  • Data Quality: {quality}")
        print(f"  • Reliability: {reliability}")

        # Investment Implications
        print(f"\n💡 INVESTMENT IMPLICATIONS:")
        if combined > 0.2 and sentiment_data['confidence'] > 0.6:
            print("  • 🟢 Positive sentiment may support price momentum")
            print("  • Consider: Sentiment-driven buying pressure possible")
        elif combined < -0.2 and sentiment_data['confidence'] > 0.6:
            print("  • 🔴 Negative sentiment may create selling pressure")
            print("  • Consider: Potential contrarian opportunity or avoid")
        else:
            print("  • 🟡 Mixed/neutral sentiment suggests normal trading")
            print("  • Consider: Focus on fundamentals over sentiment")

        # Divergence Analysis
        news_reddit_divergence = abs(sentiment_data['news_sentiment'] - sentiment_data['reddit_sentiment'])
        if news_reddit_divergence > 0.5:
            print(f"\n⚠️ SENTIMENT DIVERGENCE DETECTED:")
            print(f"  • News vs Reddit Gap: {news_reddit_divergence:.2f}")
            if sentiment_data['news_sentiment'] > sentiment_data['reddit_sentiment']:
                print("  • Professional media more positive than retail investors")
                print("  • May indicate: Institutional confidence vs retail caution")
            else:
                print("  • Retail investors more positive than professional media")
                print("  • May indicate: Retail enthusiasm vs institutional caution")

print(f"\n{'='*70}")
print("✅ Sentiment Analysis Demo Complete!")
print(f"💭 Sentiment data cached for {len(CACHE.get('sentiment', {}))} stocks")
print(f"📊 Ready for portfolio allocation with sentiment integration")
print("="*70)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



💭 DEMO: MULTI-SOURCE SENTIMENT ANALYSIS

🎯 Analyzing sentiment for: TSLA,NVDA
These stocks typically have high social engagement and news coverage
--------------------------------------------------

📊 RUNNING COMPREHENSIVE SENTIMENT ANALYSIS...
💭 Analyzing sentiment for TSLA (Tesla)...
🔍 Searching Reddit for $TSLA mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



💭 Analyzing sentiment for NVDA (Nvidia)...
🔍 Searching Reddit for $NVDA mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




SENTIMENT ANALYSIS RESULTS:
Multi-Source Sentiment Analysis:
TSLA: 🟡 Neutral ⚡ | Combined=+0.14 (News=+0.00, Reddit=+0.48) | Sources: 10 articles, 10 posts
NVDA: 🟡 Neutral ⚡ | Combined=+0.29 (News=+0.20, Reddit=+0.51) | Sources: 10 articles, 13 posts

🔍 DETAILED SENTIMENT BREAKDOWN

💭 TSLA - Tesla Sentiment Deep Dive

🎯 OVERALL SENTIMENT SUMMARY:
  • Combined Score: +0.144 (🟡 Mild Bullish Sentiment)
  • Confidence Level: 68.4%
  • Data Quality: ⚡ Medium

📰 NEWS SENTIMENT ANALYSIS:
  • News Score: +0.000
  • Articles Analyzed: 10
  • News Confidence: 36.8%
  • Sample Headlines:
    1. ➡️ Spit On, Sworn At, and Undeterred: What It’s Like to Own a Cybertruck...
    2. 📉 There's No 'AI Bubble', Says Yahoo Finance Executive Editor...
       Signals: +0 positive, -1 negative
    3. ➡️ Tesla Cybertruck sales are stalling...

📱 REDDIT SENTIMENT ANALYSIS:
  • Reddit Score: +0.480
  • Posts Found: 10
  • Total Karma: 2,152
  • Reddit Confidence: 100.0%
  • Popular Reddit Posts:
    1. 🔥 r/walls

## 📅 Tool 3 - Economic Calendar Integration

### Event-Aware Investment Analysis

This tool provides sophisticated economic calendar integration:
- **Federal Reserve Events**: FOMC meetings, interest rate decisions, Fed speeches
- **Economic Indicators**: Jobs reports, CPI, GDP, retail sales data
- **Earnings Calendar**: Quarterly earnings announcements
- **Impact Scoring**: Quantitative assessment of event impact on specific stocks
- **Risk-Adjusted Positioning**: Automatic position sizing based on upcoming events

**Key Innovation**: Integrates economic event timing into portfolio allocation decisions for more intelligent risk management.

In [9]:
# ===================================================================
# Real Economic Data Integration - FRED
# ===================================================================

import requests
from datetime import datetime, timedelta
import pandas as pd
from typing import Dict, List, Optional
import json

# Load API keys from Colab secrets
from google.colab import userdata

FRED_API_KEY = userdata.get('FRED_API_KEY')

class RealEconomicDataProvider:
    """Real economic data provider using FRED"""

    def __init__(self):
        self.fred_base_url = "https://api.stlouisfed.org/fred"

    def get_fred_data(self, series_id: str, limit: int = 10) -> List[Dict]:
        """Fetch data from FRED API"""
        try:
            url = f"{self.fred_base_url}/series/observations"
            params = {
                'series_id': series_id,
                'api_key': FRED_API_KEY,
                'file_type': 'json',
                'limit': limit,
                'sort_order': 'desc'
            }

            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                return data.get('observations', [])
            else:
                print(f"FRED API error for {series_id}: {response.status_code}")
                return []

        except Exception as e:
            print(f"Error fetching FRED data for {series_id}: {e}")
            return []

    def get_fed_interest_rates(self) -> Dict:
        """Get current and historical Fed interest rates"""
        # Federal funds rate (FEDFUNDS)
        fed_rates = self.get_fred_data('FEDFUNDS', limit=12)  # Last 12 months

        if fed_rates:
            current_rate = None
            for rate_data in fed_rates:
                if rate_data.get('value') != '.':  # FRED uses '.' for missing values
                    current_rate = float(rate_data['value'])
                    break

            return {
                'current_rate': current_rate,
                'historical': fed_rates,
                'last_updated': fed_rates[0]['date'] if fed_rates else None
            }
        return {'current_rate': None, 'historical': [], 'last_updated': None}

    def get_key_economic_indicators(self) -> Dict[str, any]:
        """Fetch key economic indicators from FRED"""
        indicators = {
            'unemployment_rate': 'UNRATE',     # Unemployment Rate
            'cpi_inflation': 'CPIAUCSL',       # Consumer Price Index
            'gdp_growth': 'GDP',               # Gross Domestic Product
            'retail_sales': 'RSXFS',           # Retail Sales
            'industrial_production': 'INDPRO', # Industrial Production
            'consumer_sentiment': 'UMCSENT'    # Consumer Sentiment
        }

        results = {}
        for indicator, series_id in indicators.items():
            data = self.get_fred_data(series_id, limit=3)
            if data:
                # Get most recent value
                latest_value = None
                for obs in data:
                    if obs.get('value') != '.':
                        latest_value = float(obs['value'])
                        break

                results[indicator] = {
                    'value': latest_value,
                    'date': data[0]['date'] if data else None,
                    'series_id': series_id,
                    'recent_data': data
                }
            else:
                results[indicator] = {'value': None, 'date': None, 'series_id': series_id}

        return results

# Initialize the real economic data provider
economic_data_provider = RealEconomicDataProvider()

print("✅ Real Economic Data Provider initialized!")
print(f"📊 FRED API: {'Configured' if FRED_API_KEY else 'Missing'}")

✅ Real Economic Data Provider initialized!
📊 FRED API: Configured


In [10]:
# ===================================
# FRED Economic Analysis Tool
# ===================================

def analyze_fred_economic_conditions(days_ahead=30):
    """Analyze current economic conditions and trends using FRED data"""

    current_indicators = economic_data_provider.get_key_economic_indicators()
    fed_data = economic_data_provider.get_fed_interest_rates()

    analysis = {
        'current_conditions': {},
        'trend_analysis': {},
        'risk_indicators': [],
        'sector_implications': {}
    }

    # Analyze each indicator's trend and volatility
    for indicator, data in current_indicators.items():
        if data.get('recent_data'):
            recent_values = []
            for obs in data['recent_data']:
                if obs.get('value') != '.':
                    recent_values.append(float(obs['value']))

            if len(recent_values) >= 2:
                # Calculate trend (simple month-over-month change)
                latest = recent_values[0]
                previous = recent_values[1]
                change = ((latest - previous) / previous) * 100 if previous != 0 else 0

                analysis['current_conditions'][indicator] = {
                    'current_value': latest,
                    'previous_value': previous,
                    'change_pct': change,
                    'trend_direction': 'increasing' if change > 0 else 'decreasing',
                    'last_updated': data['date']
                }

                # Identify concerning trends
                if indicator == 'unemployment_rate' and change > 0.5:
                    analysis['risk_indicators'].append(f"Unemployment rising: +{change:.1f}%")
                elif indicator == 'cpi_inflation' and latest > 300:  # Adjust threshold as needed
                    analysis['risk_indicators'].append(f"High inflation levels: {latest:.1f}")
                elif indicator == 'consumer_sentiment' and latest < 70:
                    analysis['risk_indicators'].append(f"Low consumer confidence: {latest:.1f}")

    return analysis

def fred_economic_tool(tickers: str = "", days_ahead: str = "30") -> str:
    """
    Economic analysis using FRED data - focuses on current conditions and trends
    """
    try:
        ticker_list = parse_tickers_input(tickers) if tickers else []

        # Get current economic analysis
        econ_analysis = analyze_fred_economic_conditions()
        fed_data = economic_data_provider.get_fed_interest_rates()

        if not ticker_list:
            # General economic conditions report
            result = f"## 📊 Current Economic Conditions Analysis (FRED Data)\n\n"
            result += f"**Analysis Date**: {datetime.now().strftime('%Y-%m-%d')}\n"
            result += f"**Data Source**: Federal Reserve Economic Data (FRED)\n\n"

            # Federal Reserve Status
            result += "### 🏦 Federal Reserve Policy\n"
            if fed_data['current_rate']:
                result += f"- **Current Fed Funds Rate**: {fed_data['current_rate']:.2f}%\n"
                result += f"- **Last Updated**: {fed_data['last_updated']}\n"

                # Rate trend analysis
                if len(fed_data['historical']) >= 2:
                    recent_rates = []
                    for rate_data in fed_data['historical'][:3]:
                        if rate_data.get('value') != '.':
                            recent_rates.append(float(rate_data['value']))

                    if len(recent_rates) >= 2:
                        rate_change = recent_rates[0] - recent_rates[1]
                        if rate_change > 0:
                            result += f"- **Recent Trend**: Rising (+{rate_change:.2f}%)\n"
                        elif rate_change < 0:
                            result += f"- **Recent Trend**: Falling ({rate_change:.2f}%)\n"
                        else:
                            result += f"- **Recent Trend**: Stable\n"

            # Current Economic Indicators
            result += "\n### 📈 Key Economic Indicators\n"
            for indicator, data in econ_analysis['current_conditions'].items():
                indicator_name = indicator.replace('_', ' ').title()
                current = data['current_value']
                change = data['change_pct']
                trend_emoji = "📈" if change > 0 else "📉" if change < 0 else "➡️"

                if 'rate' in indicator.lower() or 'unemployment' in indicator.lower():
                    result += f"- **{indicator_name}**: {current:.1f}% {trend_emoji} ({change:+.1f}%)\n"
                elif current > 1000:
                    result += f"- **{indicator_name}**: {current:,.1f} {trend_emoji} ({change:+.1f}%)\n"
                else:
                    result += f"- **{indicator_name}**: {current:.1f} {trend_emoji} ({change:+.1f}%)\n"

            # Risk Indicators
            if econ_analysis['risk_indicators']:
                result += "\n### ⚠️ Economic Risk Signals\n"
                for risk in econ_analysis['risk_indicators']:
                    result += f"- {risk}\n"
            else:
                result += "\n### ✅ No Major Economic Warning Signals\n"

            # Economic Environment Assessment
            result += "\n### 🎯 Investment Environment Assessment\n"

            # Assess based on current conditions
            conditions = econ_analysis['current_conditions']

            if 'unemployment_rate' in conditions:
                unemployment = conditions['unemployment_rate']['current_value']
                if unemployment < 4.0:
                    result += "- **Labor Market**: Strong - supports consumer spending\n"
                elif unemployment > 6.0:
                    result += "- **Labor Market**: Weak - may pressure consumer discretionary stocks\n"
                else:
                    result += "- **Labor Market**: Moderate - stable employment conditions\n"

            if fed_data['current_rate']:
                rate = fed_data['current_rate']
                if rate > 5.0:
                    result += "- **Interest Rate Environment**: High rates favor financials, pressure growth stocks\n"
                elif rate < 2.0:
                    result += "- **Interest Rate Environment**: Low rates favor growth stocks\n"
                else:
                    result += "- **Interest Rate Environment**: Moderate rates - balanced environment\n"

            result += "\n**Note**: This analysis uses current FRED data and recent trends. "
            result += "No forward-looking event calendar is included.\n"

            return result

        # Stock-specific analysis based on current economic conditions
        results = []

        for ticker in ticker_list:
            sector = get_stock_sector(ticker)

            # Assess current economic impact on this sector
            risk_score = 0.0
            recommendations = []

            # Federal Reserve impact
            if fed_data['current_rate']:
                rate = fed_data['current_rate']
                if sector.lower() == 'financials' and rate > 4.0:
                    risk_score -= 0.3  # Negative risk (good for financials)
                    recommendations.append("High rates benefit financial margins")
                elif sector.lower() == 'technology' and rate > 5.0:
                    risk_score += 0.5  # Higher risk for tech
                    recommendations.append("High rates pressure growth valuations")

            # Unemployment impact
            conditions = econ_analysis['current_conditions']
            if 'unemployment_rate' in conditions:
                unemployment = conditions['unemployment_rate']['current_value']
                change = conditions['unemployment_rate']['change_pct']

                if sector.lower() == 'consumer_discretionary':
                    if unemployment > 5.0 or change > 0.3:
                        risk_score += 0.4
                        recommendations.append("Rising unemployment pressures consumer spending")
                    elif unemployment < 4.0:
                        risk_score -= 0.2
                        recommendations.append("Low unemployment supports consumer spending")

            # Store in cache
            if 'economic_calendar' not in CACHE:
                CACHE['economic_calendar'] = {}

            CACHE['economic_calendar'][ticker] = {
                'risk_score': abs(risk_score),  # Convert to positive scale
                'high_impact_events': [],  # No future events in FRED approach
                'recommendations': recommendations if recommendations else ["Monitor current economic trends"],
                'risk_windows': [],  # No specific windows without event calendar
                'sector_exposure': sector.lower().replace(' ', '_'),
                'total_events': 0,  # No events tracked
                'current_economic_context': econ_analysis['current_conditions'],
                'fed_rate_context': fed_data,
                'analysis_date': datetime.now().isoformat(),
                'data_source': 'FRED (Current Conditions)'
            }

            # Format result
            risk_level = ("🟡 Moderate" if abs(risk_score) > 0.3 else "🟢 Low")

            result_text = f"{ticker} ({TICKER_NAMES.get(ticker, ticker)}): Current Economic Risk {risk_level}"
            result_text += f" | Sector: {sector}"

            if fed_data['current_rate']:
                result_text += f" | Fed Rate Impact: {fed_data['current_rate']:.2f}%"

            results.append(result_text)

        return "FRED Economic Conditions Analysis:\n" + "\n".join(results)

    except Exception as e:
        return f"❌ FRED Economic Analysis Error: {str(e)}"

# Update the tool
economic_calendar_tool = Tool(
    name="analyze_economic_calendar",
    func=fred_economic_tool,
    description="Analyze current economic conditions and trends using real-time FRED data. Input: ticker symbols (optional) and days ahead (unused in FRED mode)"
)

print("✅ FRED Economic Analysis Tool ready!")
print("📊 Focuses on current conditions and recent trends")
print("🔄 No hardcoded future events - pure data-driven analysis")

✅ FRED Economic Analysis Tool ready!
📊 Focuses on current conditions and recent trends
🔄 No hardcoded future events - pure data-driven analysis


## 📅 DEMO: Economic Calendar Analysis

This demonstration shows how economic events are integrated into investment analysis:

1. **Federal Reserve Events**: FOMC meetings and rate decisions
2. **Economic Indicators**: Jobs reports, inflation data, GDP releases
3. **Earnings Calendar**: Company-specific quarterly reports
4. **Impact Scoring**: Quantitative assessment of event significance
5. **Risk Windows**: Periods of expected high volatility
6. **Sector-Specific Analysis**: How events affect different market sectors

**Test Scenario**: Analyze economic calendar impact for tech and financial stocks

In [11]:
# ===================================================================
# DEMO: FRED Economic Analysis
# ===================================================================

print("="*70)
print("📊 DEMO: FRED ECONOMIC CONDITIONS ANALYSIS")
print("="*70)

# Test economic analysis with mixed sector stocks
test_economic_tickers = "AAPL,JPM,TSLA"
print(f"\n🎯 Analyzing current economic conditions for: {test_economic_tickers}")
print("Representing: Technology, Financials, Consumer Discretionary sectors")
print("📊 Using real-time FRED data")
print("-"*50)

print("\n1️⃣ CURRENT ECONOMIC CONDITIONS (FRED Data):")
print("-"*50)

# Test the FRED economic data provider first
try:
    # Show current economic indicators from FRED
    current_indicators = economic_data_provider.get_key_economic_indicators()
    fed_data = economic_data_provider.get_fed_interest_rates()

    print("📈 CURRENT ECONOMIC SNAPSHOT:")
    print(f"   • Federal Funds Rate: {fed_data['current_rate']:.2f}%" if fed_data['current_rate'] else "   • Federal Funds Rate: Data loading...")

    for indicator, data in current_indicators.items():
        if data['value'] is not None:
            indicator_name = indicator.replace('_', ' ').title()
            if 'rate' in indicator.lower() or 'unemployment' in indicator.lower():
                print(f"   • {indicator_name}: {data['value']:.1f}% (as of {data['date']})")
            elif data['value'] > 1000:  # Large numbers like GDP, CPI
                print(f"   • {indicator_name}: {data['value']:,.1f} (as of {data['date']})")
            else:
                print(f"   • {indicator_name}: {data['value']:.1f} (as of {data['date']})")

    print("\n")

except Exception as e:
    print(f"⚠️ Economic data loading: {str(e)}")
    print("Proceeding with available data sources...\n")

# Use the FRED economic analysis tool
general_conditions = fred_economic_tool("", "30")
print(general_conditions)

print(f"\n2️⃣ STOCK-SPECIFIC ECONOMIC CONDITIONS ANALYSIS:")
print("-"*50)
specific_analysis = fred_economic_tool(test_economic_tickers, "30")
print(specific_analysis)

print(f"\n3️⃣ DETAILED FRED ECONOMIC BREAKDOWN:")
print("="*50)

for ticker in test_economic_tickers.split(','):
    ticker = ticker.strip()
    if ticker in CACHE.get('economic_calendar', {}):
        print(f"\n{'='*60}")
        print(f"📊 {ticker} - {TICKER_NAMES.get(ticker, ticker)} Economic Conditions Analysis")
        print(f"{'='*60}")

        econ_data = CACHE['economic_calendar'][ticker]

        # Economic Risk Assessment (FRED-based)
        print(f"\n🎯 CURRENT ECONOMIC RISK ASSESSMENT:")
        risk_score = econ_data.get('risk_score', 0)
        print(f"  • Economic Risk Score: {risk_score:.2f}")
        print(f"  • Data Source: {econ_data.get('data_source', 'FRED')}")

        if risk_score > 0.5:
            risk_level = "🟡 MODERATE RISK"
            advice = "Monitor current economic trends closely"
        elif risk_score > 0.3:
            risk_level = "🟡 LOW-MODERATE RISK"
            advice = "Standard positioning with awareness of conditions"
        else:
            risk_level = "🟢 LOW RISK"
            advice = "Favorable current economic environment"

        print(f"  • Risk Level: {risk_level}")
        print(f"  • Recommendation: {advice}")
        print(f"  • Sector Exposure: {econ_data.get('sector_exposure', 'Unknown').replace('_', ' ').title()}")

        # Show current economic context
        if 'current_economic_context' in econ_data:
            context = econ_data['current_economic_context']
            active_indicators = len([k for k, v in context.items() if v.get('value') is not None])
            print(f"  • Current Indicators Tracked: {active_indicators}")

        # Show Fed rate context
        if 'fed_rate_context' in econ_data:
            fed_context = econ_data['fed_rate_context']
            if fed_context.get('current_rate'):
                print(f"  • Current Fed Funds Rate: {fed_context['current_rate']:.2f}%")

        # Current Economic Conditions Impact (instead of future events)
        current_conditions_impact = econ_data.get('current_conditions_impact', {})
        if current_conditions_impact:
            print(f"\n📈 CURRENT CONDITIONS IMPACT:")
            for condition, impact_data in current_conditions_impact.items():
                condition_name = condition.replace('_', ' ').title()
                print(f"  • {condition_name}:")
                print(f"    Current Level: {impact_data.get('current', 'N/A')}")
                print(f"    Trend: {impact_data.get('trend', 'stable').title()}")
                print(f"    Impact: {impact_data.get('impact', 'neutral').title()}")

        # Economic Recommendations (FRED-based)
        recommendations = econ_data.get('recommendations', [])
        if recommendations:
            print(f"\n💡 CURRENT CONDITIONS RECOMMENDATIONS:")
            for i, rec in enumerate(recommendations, 1):
                print(f"  {i}. {rec}")

        # Portfolio Allocation based on Current Conditions
        print(f"\n📊 PORTFOLIO ALLOCATION BASED ON CURRENT CONDITIONS:")
        sector = econ_data.get('sector_exposure', '').replace('_', ' ')

        if risk_score > 0.4:
            allocation_advice = f"Consider defensive positioning in {ticker} due to current economic headwinds"
        elif risk_score < 0.2:
            allocation_advice = f"Current conditions support standard {ticker} allocation"
        else:
            allocation_advice = f"Maintain {ticker} allocation with monitoring of economic trends"

        print(f"  • Allocation Guidance: {allocation_advice}")

        # Fed Rate Environment Impact
        if 'fed_rate_context' in econ_data and econ_data['fed_rate_context'].get('current_rate'):
            fed_rate = econ_data['fed_rate_context']['current_rate']
            if fed_rate > 5.0:
                print(f"  • Rate Environment: High rates ({fed_rate:.2f}%) - assess sector impact")
            elif fed_rate < 2.0:
                print(f"  • Rate Environment: Low rates ({fed_rate:.2f}%) - growth favorable")
            else:
                print(f"  • Rate Environment: Moderate rates ({fed_rate:.2f}%) - balanced conditions")

        # Current Economic Trends (instead of future timing)
        print(f"\n⏰ CURRENT TREND ANALYSIS:")
        if 'current_economic_context' in econ_data:
            context = econ_data['current_economic_context']

            unemployment_trend = None
            if 'unemployment_rate' in context:
                change = context['unemployment_rate'].get('change_pct', 0)
                if change > 0.2:
                    unemployment_trend = "Rising unemployment may pressure consumer sectors"
                elif change < -0.2:
                    unemployment_trend = "Falling unemployment supports economic growth"
                else:
                    unemployment_trend = "Stable unemployment conditions"

            if unemployment_trend:
                print(f"  • Labor Market: {unemployment_trend}")

            print(f"  • Overall: Current conditions suggest {risk_level.lower()} for {ticker}")

        # Sector-Specific Current Conditions Analysis
        print(f"\n🏭 SECTOR-SPECIFIC CURRENT CONDITIONS ({sector.title()}):")

        sector_insights = {
            'technology': "Tech stocks currently affected by interest rates and growth expectations",
            'financials': "Financial stocks currently impacted by rate environment and economic stability",
            'consumer discretionary': "Consumer stocks currently influenced by employment and sentiment levels",
            'healthcare': "Healthcare currently showing defensive characteristics",
            'energy': "Energy stocks currently responding to inflation and economic activity"
        }

        insight = sector_insights.get(sector.lower(), "Sector responding to current economic conditions")
        print(f"  • Current Sector Environment: {insight}")

        # Real-time data freshness
        analysis_date = econ_data.get('analysis_date')
        if analysis_date:
            try:
                from datetime import datetime
                parsed_date = datetime.fromisoformat(analysis_date.replace('Z', '+00:00'))
                print(f"  • Analysis Date: {parsed_date.strftime('%Y-%m-%d %H:%M:%S')}")
            except:
                print(f"  • Analysis Date: {analysis_date}")

    else:
        print(f"\n⚠️ No economic analysis data available for {ticker}")
        print(f"   Run the FRED economic analysis tool first")

print(f"\n{'='*70}")
print("✅ FRED Economic Analysis Demo Complete!")

# Summary
economic_cache = CACHE.get('economic_calendar', {})
print(f"📊 Economic analysis cached for {len(economic_cache)} stocks")

if economic_cache:
    print(f"📈 Data source: FRED (Federal Reserve Economic Data)")

    # Check data quality
    sample_data = next(iter(economic_cache.values()))
    data_sources = sample_data.get('data_source', 'Unknown')
    fed_data_available = sample_data.get('fed_rate_context', {}).get('current_rate') is not None
    econ_indicators_available = len(sample_data.get('current_economic_context', {}))

    print(f"\n📈 DATA QUALITY SUMMARY:")
    print(f"  • Data Integration: {data_sources}")
    print(f"  • Fed Rate Data: {'✅ Live' if fed_data_available else '❌ Unavailable'}")
    print(f"  • Economic Indicators: {econ_indicators_available} metrics active")
    print(f"  • Analysis Type: Current conditions and trends")
else:
    print(f"⚠️ No economic analysis data in cache")
    print(f"   Run: fred_economic_tool('{test_economic_tickers}', '30')")

print(f"🎯 FRED analysis ready for portfolio optimization")
print("="*70)

📊 DEMO: FRED ECONOMIC CONDITIONS ANALYSIS

🎯 Analyzing current economic conditions for: AAPL,JPM,TSLA
Representing: Technology, Financials, Consumer Discretionary sectors
📊 Using real-time FRED data
--------------------------------------------------

1️⃣ CURRENT ECONOMIC CONDITIONS (FRED Data):
--------------------------------------------------
📈 CURRENT ECONOMIC SNAPSHOT:
   • Federal Funds Rate: 4.22%
   • Unemployment Rate: 4.3% (as of 2025-08-01)
   • Cpi Inflation: 323.4 (as of 2025-08-01)
   • Gdp Growth: 30,485.7 (as of 2025-04-01)
   • Retail Sales: 632,490.0 (as of 2025-08-01)
   • Industrial Production: 103.9 (as of 2025-08-01)
   • Consumer Sentiment: 58.2 (as of 2025-08-01)


## 📊 Current Economic Conditions Analysis (FRED Data)

**Analysis Date**: 2025-10-19
**Data Source**: Federal Reserve Economic Data (FRED)

### 🏦 Federal Reserve Policy
- **Current Fed Funds Rate**: 4.22%
- **Last Updated**: 2025-09-01
- **Recent Trend**: Falling (-0.11%)

### 📈 Key Economic Indicators

## 💼 Tool 4 - Portfolio Allocator with Economic Calendar

### Intelligent Risk-Aware Portfolio Construction

This is the crown jewel of our system, combining:
- **Multi-Factor Scoring**: Technical + Fundamental + Sentiment analysis
- **Economic Calendar Integration**: Position sizing adjustments based on upcoming events
- **Risk Profile Adaptation**: Conservative, Moderate, and Aggressive strategies
- **Dynamic Rebalancing**: Automatic adjustments for economic event risk

**Key Innovation**: Risk profiles now include "economic sensitivity" - how much weight to give upcoming events when sizing positions.

**Weighting Schemes**:
- **Conservative**: 50% fundamentals, 30% technical, 20% sentiment + 80% economic sensitivity
- **Moderate**: 40% fundamentals, 35% technical, 25% sentiment + 60% economic sensitivity  
- **Aggressive**: 30% fundamentals, 35% technical, 35% sentiment + 40% economic sensitivity

In [12]:
# ===================================================================
# Tool 4: Portfolio Allocator with FRED Economic Conditions Integration
# ===================================================================

def portfolio_allocator(parameters: str) -> str:
    """
    Create sophisticated portfolio allocation integrating current economic conditions analysis.

    This allocator considers:
    1. Multi-factor scoring (technical + fundamental + sentiment)
    2. Current economic conditions risk adjustments
    3. Risk profile-appropriate sensitivity to economic environment
    4. Intelligent position sizing based on current conditions
    """
    try:
        # Parse input parameters
        params = {}
        for item in parameters.split(','):
            if '=' in item:
                key, value = item.split('=', 1)
                params[key.strip()] = value.strip()

        budget = float(params.get('budget', 10000))
        risk = params.get('risk', 'moderate').lower()

        # Risk profiles with economic conditions integration
        risk_profiles = {
            'conservative': {
                'num_stocks': 8,
                'fundamental_weight': 0.50,
                'technical_weight': 0.30,
                'sentiment_weight': 0.20,
                'max_position': 0.15,
                'economic_sensitivity': 0.8,  # High sensitivity to economic conditions
                'description': 'Safety-focused with high economic awareness'
            },
            'moderate': {
                'num_stocks': 6,
                'fundamental_weight': 0.40,
                'technical_weight': 0.35,
                'sentiment_weight': 0.25,
                'max_position': 0.20,
                'economic_sensitivity': 0.6,  # Balanced economic consideration
                'description': 'Balanced approach with moderate economic sensitivity'
            },
            'aggressive': {
                'num_stocks': 4,
                'fundamental_weight': 0.30,
                'technical_weight': 0.35,
                'sentiment_weight': 0.35,
                'max_position': 0.25,
                'economic_sensitivity': 0.4,  # Lower sensitivity, higher risk tolerance
                'description': 'Growth-focused, accepting economic volatility'
            }
        }

        profile = risk_profiles.get(risk, risk_profiles['moderate'])

        # Validate required data availability
        required_data = ['market_data', 'sentiment']
        missing_data = [data for data in required_data if data not in CACHE or not CACHE[data]]

        if missing_data:
            return f"Required data missing: {', '.join(missing_data)}. Please run market data and sentiment analysis first."

        # Check for economic conditions data (optional but enhances analysis)
        has_economic_data = 'economic_calendar' in CACHE and CACHE['economic_calendar']

        # Score all available stocks
        stock_scores = []
        available_tickers = set(CACHE['market_data'].keys()) & set(CACHE['sentiment'].keys())

        # Economic conditions summary for reporting
        economic_summary = {
            'high_risk_stocks': [],
            'average_risk_score': 0,
            'conditions_tracked': 0
        }

        for ticker in available_tickers:
            try:
                market = CACHE['market_data'][ticker]
                sentiment = CACHE['sentiment'][ticker]

                # Ensure we have required price data
                current_price = market.get('current_price')
                if not current_price or current_price <= 0:
                    continue

                # Normalize all component scores to 0-1 range for fair weighting
                sharpe = market.get('sharpe', 0)
                sharpe_score = max(0, min(1, (sharpe + 1) / 3))

                technical_score = max(0, min(1, (market.get('technical_score', 0) + 3) / 6))
                fundamental_score = max(0, min(1, market.get('fundamental_score', 0) / 4))
                sentiment_score = max(0, min(1, (sentiment.get('score', 0) + 1) / 2))

                # Calculate base combined score using profile weights
                base_combined_score = (
                    profile['fundamental_weight'] * fundamental_score +
                    profile['technical_weight'] * technical_score +
                    profile['sentiment_weight'] * sentiment_score
                )

                # Economic Conditions Risk Adjustment (FRED-based)
                economic_risk_adjustment = 1.0  # Default: no adjustment
                economic_risk_score = 0.0
                conditions_details = "No economic conditions analysis available"

                if has_economic_data and ticker in CACHE['economic_calendar']:
                    econ_data = CACHE['economic_calendar'][ticker]
                    economic_risk_score = econ_data.get('risk_score', 0.0)

                    # Calculate adjustment based on current economic conditions
                    risk_penalty = economic_risk_score * profile['economic_sensitivity']

                    # Apply graduated risk penalties based on current conditions
                    if economic_risk_score > 0.5:  # Higher economic risk
                        economic_risk_adjustment = max(0.7, 1.0 - (risk_penalty * 0.3))
                        economic_summary['high_risk_stocks'].append(ticker)
                        conditions_details = f"Elevated economic conditions risk (score: {economic_risk_score:.2f})"
                    elif economic_risk_score > 0.3:  # Moderate economic risk
                        economic_risk_adjustment = max(0.85, 1.0 - (risk_penalty * 0.2))
                        conditions_details = f"Moderate economic conditions risk"
                    else:  # Low economic risk
                        conditions_details = "Favorable current economic conditions"

                    economic_summary['conditions_tracked'] += 1

                # Apply economic conditions adjustment to final score
                final_adjusted_score = base_combined_score * economic_risk_adjustment

                # Store comprehensive scoring data
                stock_scores.append({
                    'ticker': ticker,
                    'base_score': base_combined_score,
                    'adjusted_score': final_adjusted_score,
                    'economic_adjustment': economic_risk_adjustment,
                    'economic_risk': economic_risk_score,
                    'conditions_details': conditions_details,

                    # Component scores for transparency
                    'fundamental': fundamental_score,
                    'technical': technical_score,
                    'sentiment': sentiment_score,
                    'sharpe': sharpe,

                    # Trading data
                    'price': current_price,
                    'volatility': market.get('volatility', 0.15),
                    'beta': market.get('beta', 1.0),
                    'sector': get_stock_sector(ticker)
                })

            except Exception as e:
                print(f"Warning: Skipping {ticker} due to error: {e}")
                continue

        if not stock_scores:
            return "No valid stocks found with complete data. Please check data availability."

        # Select top stocks based on ADJUSTED scores (economic conditions integrated)
        stock_scores.sort(key=lambda x: x['adjusted_score'], reverse=True)
        selected_stocks = stock_scores[:min(profile['num_stocks'], len(stock_scores))]

        # Calculate portfolio allocations
        total_adjusted_score = sum(s['adjusted_score'] for s in selected_stocks)

        if total_adjusted_score <= 0:
            return "Total adjusted score is zero. Cannot create meaningful allocation."

        # Build comprehensive allocation report
        result = f"## Portfolio Allocation (Economic Conditions Integrated)\n\n"
        result += f"### Portfolio Configuration\n"
        result += f"- **Budget**: ${budget:,.0f}\n"
        result += f"- **Risk Profile**: {risk.title()} ({profile['description']})\n"
        result += f"- **Factor Weights**: Fundamental {profile['fundamental_weight']:.0%}, "
        result += f"Technical {profile['technical_weight']:.0%}, Sentiment {profile['sentiment_weight']:.0%}\n"
        result += f"- **Economic Sensitivity**: {profile['economic_sensitivity']:.0%}\n\n"

        # Economic Conditions Integration Summary
        if has_economic_data:
            avg_economic_risk = sum(s['economic_risk'] for s in selected_stocks) / len(selected_stocks)
            economic_summary['average_risk_score'] = avg_economic_risk

            result += f"### Current Economic Conditions Integration\n"
            result += f"- **Conditions Analyzed**: {economic_summary['conditions_tracked']} stocks with economic data\n"
            result += f"- **Average Conditions Risk**: {avg_economic_risk:.2f}\n"
            result += f"- **Elevated Risk Stocks**: {len(economic_summary['high_risk_stocks'])}/{len(selected_stocks)}\n"
            if economic_summary['high_risk_stocks']:
                result += f"- **Stocks with Economic Risk**: {', '.join(economic_summary['high_risk_stocks'])}\n"
            result += "\n"
        else:
            result += f"### Economic Conditions Analysis Not Available\n"
            result += f"Run FRED economic analysis for risk-aware allocation.\n\n"

        # Detailed Allocation Table
        result += f"### Portfolio Allocation Details\n\n"
        result += "| Stock | Weight | Amount | Shares | Base Score | Econ Adj | Final Score | Risk Level |\n"
        result += "|-------|--------|--------|--------|-----------|----------|-------------|------------|\n"

        allocations = []
        portfolio_metrics = {
            'total_allocation': 0,
            'weighted_sharpe': 0,
            'weighted_volatility': 0,
            'weighted_beta': 0,
            'sector_diversity': set()
        }

        for stock in selected_stocks:
            # Calculate allocation weight with position size limits
            raw_weight = stock['adjusted_score'] / total_adjusted_score
            capped_weight = min(raw_weight, profile['max_position'])

            # Renormalize to ensure weights sum to ~1
            total_capped = sum(min(s['adjusted_score'] / total_adjusted_score, profile['max_position'])
                             for s in selected_stocks)
            final_weight = capped_weight / total_capped if total_capped > 0 else 0

            allocation_amount = budget * final_weight
            shares = int(allocation_amount / stock['price']) if stock['price'] > 0 else 0

            # Risk level indicator based on economic conditions
            if stock['economic_risk'] > 0.5:
                risk_emoji = "🟡"
            elif stock['economic_risk'] > 0.3:
                risk_emoji = "🟡"
            else:
                risk_emoji = "🟢"

            # Add to allocation table
            result += f"| {stock['ticker']} | {final_weight:.1%} | ${allocation_amount:,.0f} | {shares} | "
            result += f"{stock['base_score']:.3f} | {stock['economic_adjustment']:.2f} | "
            result += f"{stock['adjusted_score']:.3f} | {risk_emoji} |\n"

            # Store allocation for portfolio metrics
            allocations.append({
                'ticker': stock['ticker'],
                'weight': final_weight,
                'amount': allocation_amount,
                'shares': shares,
                'sector': stock['sector']
            })

            # Update portfolio metrics
            portfolio_metrics['total_allocation'] += final_weight
            portfolio_metrics['weighted_sharpe'] += stock['sharpe'] * final_weight
            portfolio_metrics['weighted_volatility'] += stock['volatility'] * final_weight
            portfolio_metrics['weighted_beta'] += stock['beta'] * final_weight
            portfolio_metrics['sector_diversity'].add(stock['sector'])

        # Portfolio Summary Statistics
        result += f"\n### Portfolio Summary Statistics\n"
        result += f"- **Total Allocation**: {portfolio_metrics['total_allocation']:.1%}\n"
        result += f"- **Expected Sharpe Ratio**: {portfolio_metrics['weighted_sharpe']:.3f}\n"
        result += f"- **Portfolio Volatility**: {portfolio_metrics['weighted_volatility']:.1%}\n"
        result += f"- **Portfolio Beta**: {portfolio_metrics['weighted_beta']:.2f}\n"
        result += f"- **Sector Diversification**: {len(portfolio_metrics['sector_diversity'])} sectors\n"
        result += f"- **Position Count**: {len(selected_stocks)}\n"
        result += f"- **Max Position Size**: {profile['max_position']:.0%}\n\n"

        # Recommendations Section
        result += f"### Investment Recommendations\n"

        # Economic conditions specific recommendations
        if has_economic_data and economic_summary['high_risk_stocks']:
            result += f"#### Economic Conditions Risk Management\n"
            result += f"- **Economic Risk Alert**: {', '.join(economic_summary['high_risk_stocks'])} show elevated economic sensitivity\n"
            result += f"- **Positioning**: Allocations adjusted based on current economic environment\n"
            result += f"- **Monitoring**: Watch for changes in economic conditions that may affect these positions\n\n"

        # Performance and risk recommendations
        result += f"#### Performance & Risk Analysis\n"
        if portfolio_metrics['weighted_sharpe'] > 1.0:
            result += f"- **Risk-Adjusted Returns**: Strong expected Sharpe ratio ({portfolio_metrics['weighted_sharpe']:.2f})\n"
        elif portfolio_metrics['weighted_sharpe'] < 0.5:
            result += f"- **Risk-Adjusted Returns**: Consider more defensive allocation\n"
        else:
            result += f"- **Risk-Adjusted Returns**: Reasonable risk-reward balance\n"

        if portfolio_metrics['weighted_volatility'] > 0.25:
            result += f"- **Volatility**: Higher portfolio volatility ({portfolio_metrics['weighted_volatility']:.1%})\n"
        elif portfolio_metrics['weighted_volatility'] < 0.15:
            result += f"- **Volatility**: Conservative volatility profile ({portfolio_metrics['weighted_volatility']:.1%})\n"

        result += f"- **Market Sensitivity**: Beta of {portfolio_metrics['weighted_beta']:.2f}\n\n"

        # Store allocation in cache
        CACHE['last_allocation'] = {
            'allocations': allocations,
            'economic_conditions_integrated': has_economic_data,
            'risk_profile': risk,
            'total_economic_risk': economic_summary.get('average_risk_score', 0),
            'economic_risk_stocks': economic_summary['high_risk_stocks'],
            'portfolio_metrics': portfolio_metrics,
            'creation_timestamp': time.time()
        }

        return result

    except Exception as e:
        return f"Portfolio allocation error: {str(e)}\n\nPlease verify all required data is available and parameters are correct."

# Create LangChain Tool
portfolio_tool = Tool(
    name="create_portfolio",
    func=portfolio_allocator,
    description="Create optimized portfolio allocation with current economic conditions integration. Input format: budget=X,risk=conservative/moderate/aggressive"
)

print("Portfolio Allocator ready!")
print("Multi-factor scoring integrated")
print("FRED economic conditions risk adjustments enabled")
print("Risk-aware position sizing active")

Portfolio Allocator ready!
Multi-factor scoring integrated
FRED economic conditions risk adjustments enabled
Risk-aware position sizing active


## 💼 DEMO: Portfolio Allocation with Economic Calendar

This comprehensive demonstration showcases the complete portfolio allocation process:

1. **Multi-Risk Profile Testing**: Conservative, Moderate, and Aggressive strategies
2. **Economic Calendar Integration**: How upcoming events affect allocation decisions
3. **Transparent Scoring**: See exactly how each stock is evaluated and weighted
4. **Risk Adjustments**: Watch position sizes change based on economic event risk
5. **Performance Comparison**: Before and after economic calendar integration

**Test Scenario**: $50,000 budget across different risk profiles with tech-focused stocks

In [13]:
# ===================================================================
# DEMO: Portfolio Allocation with FRED Economic Conditions Integration
# ===================================================================

print("="*70)
print("💰 DEMO: COMPLETE PORTFOLIO ALLOCATION SYSTEM (FRED)")
print("="*70)

# Prepare comprehensive test data
test_portfolio_tickers = "AAPL,MSFT,GOOGL,NVDA,TSLA,META,JPM,V"
test_budget = 50000

print(f"\n📊 PREPARING COMPREHENSIVE ANALYSIS...")
print(f"Selected tickers: {test_portfolio_tickers}")
print(f"Test budget: ${test_budget:,}")
print("-"*50)

# Step 1: Gather all required data
print("\n1️⃣ Gathering Market Data...")
market_result = gather_market_data_tool(test_portfolio_tickers)
print("   ✅ Market data collected")

print("\n2️⃣ Analyzing Sentiment (News + Reddit)...")
sentiment_result = combined_sentiment_tool(test_portfolio_tickers)
print("   ✅ Sentiment analysis complete")

print("\n3️⃣ FRED Economic Conditions Analysis...")
economic_result = fred_economic_tool(test_portfolio_tickers, "30")
print("   ✅ Economic conditions integrated")

# Display comprehensive scoring transparency
print(f"\n{'='*70}")
print("🎯 TRANSPARENT SCORING SYSTEM ANALYSIS")
print("="*70)

print(f"\n📊 Multi-Factor Scoring Breakdown:")
print("-"*50)

for ticker in test_portfolio_tickers.split(','):
    ticker = ticker.strip()
    if (ticker in CACHE.get('market_data', {}) and
        ticker in CACHE.get('sentiment', {})):

        market = CACHE['market_data'][ticker]
        sentiment = CACHE['sentiment'][ticker]
        economic = CACHE.get('economic_calendar', {}).get(ticker, {})

        print(f"\n📈 {ticker} - {TICKER_NAMES.get(ticker, ticker)}")
        print(f"   Sector: {get_stock_sector(ticker)}")

        # Raw component scores
        print(f"   Raw Scores:")
        print(f"     • Sharpe Ratio: {market.get('sharpe', 0):.3f}")
        print(f"     • Technical: {market.get('technical_score', 0):.2f}/3.0")
        print(f"     • Fundamental: {market.get('fundamental_score', 0):.2f}/4.0")
        print(f"     • Sentiment: {sentiment.get('score', 0):+.3f}")

        # Normalized scores (what the allocator uses)
        sharpe_norm = max(0, min(1, (market.get('sharpe', 0) + 1) / 3))
        tech_norm = max(0, min(1, (market.get('technical_score', 0) + 3) / 6))
        fund_norm = max(0, min(1, market.get('fundamental_score', 0) / 4))
        sent_norm = max(0, min(1, (sentiment.get('score', 0) + 1) / 2))

        print(f"   Normalized (0-1):")
        print(f"     • Sharpe: {sharpe_norm:.3f}")
        print(f"     • Technical: {tech_norm:.3f}")
        print(f"     • Fundamental: {fund_norm:.3f}")
        print(f"     • Sentiment: {sent_norm:.3f}")

        # Economic conditions impact
        economic_risk = economic.get('risk_score', 0)
        print(f"   Economic Conditions:")
        print(f"     • Risk Score: {economic_risk:.2f}")
        print(f"     • Data Source: {economic.get('data_source', 'N/A')}")

        # Show how different risk profiles would weight this stock
        profiles = {
            'Conservative': {'fund': 0.50, 'tech': 0.30, 'sent': 0.20, 'econ_sens': 0.8},
            'Moderate': {'fund': 0.40, 'tech': 0.35, 'sent': 0.25, 'econ_sens': 0.6},
            'Aggressive': {'fund': 0.30, 'tech': 0.35, 'sent': 0.35, 'econ_sens': 0.4}
        }

        print(f"   Combined Scores by Risk Profile:")
        for profile_name, weights in profiles.items():
            base_score = (weights['fund'] * fund_norm +
                         weights['tech'] * tech_norm +
                         weights['sent'] * sent_norm)

            # Economic adjustment (FRED-based)
            risk_penalty = economic_risk * weights['econ_sens']
            if economic_risk > 0.5:
                econ_adj = max(0.7, 1.0 - (risk_penalty * 0.3))
            elif economic_risk > 0.3:
                econ_adj = max(0.85, 1.0 - (risk_penalty * 0.2))
            else:
                econ_adj = 1.0

            final_score = base_score * econ_adj

            print(f"     • {profile_name}: {base_score:.3f} → {final_score:.3f} (adj: {econ_adj:.3f})")

# Test different risk profiles
print(f"\n{'='*70}")
print("💼 PORTFOLIO ALLOCATION COMPARISON BY RISK PROFILE")
print("="*70)

risk_profiles = ['conservative', 'moderate', 'aggressive']
allocation_results = {}

for risk_level in risk_profiles:
    print(f"\n{'='*60}")
    print(f"Testing: {risk_level.upper()} Risk Profile")
    print(f"{'='*60}")

    test_params = f"budget={test_budget},risk={risk_level}"
    print(f"Parameters: {test_params}")
    print("-"*40)

    # Run allocator
    allocation_result = portfolio_allocator(test_params)
    allocation_results[risk_level] = allocation_result
    print(allocation_result)

# Economic Conditions Impact Analysis
print(f"\n{'='*70}")
print("📊 FRED ECONOMIC CONDITIONS IMPACT ANALYSIS")
print("="*70)

print(f"\n🔍 How Current Economic Conditions Affect Allocations:")
print("-"*50)

# Show specific impact for higher-risk stocks
elevated_risk_tickers = []
if 'economic_calendar' in CACHE:
    for ticker, data in CACHE['economic_calendar'].items():
        if data.get('risk_score', 0) > 0.3:
            elevated_risk_tickers.append(ticker)

if elevated_risk_tickers:
    print(f"\n📊 Stocks with Economic Conditions Risk: {', '.join(elevated_risk_tickers)}")
    print("These stocks received adjusted allocations based on current economic environment:")

    for ticker in elevated_risk_tickers:
        if ticker in CACHE['economic_calendar']:
            econ_data = CACHE['economic_calendar'][ticker]
            print(f"\n  {ticker} ({TICKER_NAMES.get(ticker, ticker)}):")
            print(f"    • Economic Risk Score: {econ_data['risk_score']:.2f}")
            print(f"    • Data Source: {econ_data.get('data_source', 'FRED')}")

            # Show adjustment calculation for each profile
            for profile in ['Conservative', 'Moderate', 'Aggressive']:
                sensitivity = {'Conservative': 0.8, 'Moderate': 0.6, 'Aggressive': 0.4}[profile]
                risk_penalty = econ_data['risk_score'] * sensitivity

                if econ_data['risk_score'] > 0.5:
                    adjustment = max(0.7, 1.0 - (risk_penalty * 0.3))
                elif econ_data['risk_score'] > 0.3:
                    adjustment = max(0.85, 1.0 - (risk_penalty * 0.2))
                else:
                    adjustment = 1.0

                reduction = (1.0 - adjustment) * 100
                print(f"    • {profile} allocation adjustment: {reduction:.0f}% reduction" if reduction > 0 else f"    • {profile} allocation: No adjustment needed")

else:
    print(f"\n✅ No stocks showing significant economic conditions risk")
    print("Current economic environment appears favorable for standard allocations")

# Portfolio Performance Analysis
print(f"\n{'='*70}")
print("📈 PORTFOLIO PERFORMANCE & FRED INTEGRATION ANALYSIS")
print("="*70)

print(f"\n💡 Key Insights from FRED Economic Conditions Integration:")
print("-"*50)

conditions_tracked = 0
elevated_risk_count = 0

if 'economic_calendar' in CACHE:
    for ticker, data in CACHE['economic_calendar'].items():
        conditions_tracked += 1
        if data.get('risk_score', 0) > 0.3:
            elevated_risk_count += 1

print(f"📊 Analysis Coverage:")
print(f"  • Stocks Analyzed: {len(test_portfolio_tickers.split(','))}")
print(f"  • Economic Conditions Tracked: {conditions_tracked}")
print(f"  • Stocks with Elevated Risk: {elevated_risk_count}")
print(f"  • FRED Integration: {'✅ Active' if 'economic_calendar' in CACHE else '❌ Not Available'}")

print(f"\n🎯 FRED-Based Risk Management Benefits:")
print(f"  1. Current economic conditions integrated into position sizing")
print(f"  2. Real-time Fed policy and economic indicators consideration")
print(f"  3. Sector-specific economic sensitivity analysis")
print(f"  4. Transparent, data-driven decision process")
print(f"  5. Integration of 4 data sources: technical + fundamental + sentiment + FRED conditions")

# Show FRED economic context
if 'economic_calendar' in CACHE:
    sample_data = next(iter(CACHE['economic_calendar'].values()))
    if 'fed_rate_context' in sample_data:
        fed_rate = sample_data['fed_rate_context'].get('current_rate')
        if fed_rate:
            print(f"\n📈 Current Economic Environment Context:")
            print(f"  • Federal Funds Rate: {fed_rate:.2f}%")

            if fed_rate > 5.0:
                print(f"  • Environment: High rate environment favors financials, pressures growth")
            elif fed_rate < 2.0:
                print(f"  • Environment: Low rate environment supports growth stocks")
            else:
                print(f"  • Environment: Moderate rate environment - balanced conditions")

    if 'current_economic_context' in sample_data:
        context = sample_data['current_economic_context']
        unemployment = context.get('unemployment_rate', {}).get('value')
        if unemployment:
            print(f"  • Unemployment Rate: {unemployment:.1f}%")
            if unemployment < 4.5:
                print(f"  • Labor Market: Tight - supports consumer spending")
            elif unemployment > 6.0:
                print(f"  • Labor Market: Loose - may pressure consumer sectors")

# Show cached allocation data
if 'last_allocation' in CACHE:
    cached_allocation = CACHE['last_allocation']
    print(f"\n📊 Final Portfolio Metrics (from last allocation):")
    metrics = cached_allocation.get('portfolio_metrics', {})
    print(f"  • Expected Sharpe Ratio: {metrics.get('weighted_sharpe', 0):.3f}")
    print(f"  • Portfolio Volatility: {metrics.get('weighted_volatility', 0):.1%}")
    print(f"  • Portfolio Beta: {metrics.get('weighted_beta', 1.0):.2f}")
    print(f"  • Sector Diversity: {len(metrics.get('sector_diversity', []))} sectors")
    print(f"  • Economic Conditions Integrated: {cached_allocation.get('economic_conditions_integrated', False)}")

print(f"\n{'='*70}")
print("✅ Portfolio Allocation with FRED Integration Demo Complete!")
print("="*70)

print(f"\n🏆 COMPREHENSIVE SYSTEM CAPABILITIES DEMONSTRATED:")
print(f"  ✓ Real-time market data integration")
print(f"  ✓ Multi-source sentiment analysis (news + Reddit)")
print(f"  ✓ FRED economic conditions risk assessment")
print(f"  ✓ Multi-factor portfolio optimization")
print(f"  ✓ Current conditions-based position sizing")
print(f"  ✓ Transparent scoring and decision process")
print(f"  ✓ Professional-grade investment analysis with Federal Reserve data")

💰 DEMO: COMPLETE PORTFOLIO ALLOCATION SYSTEM (FRED)

📊 PREPARING COMPREHENSIVE ANALYSIS...
Selected tickers: AAPL,MSFT,GOOGL,NVDA,TSLA,META,JPM,V
Test budget: $50,000
--------------------------------------------------

1️⃣ Gathering Market Data...
📊 Processing AAPL (Apple)...
📊 Processing MSFT (Microsoft)...
📊 Processing GOOGL (Google)...
📊 Processing NVDA (Nvidia)...
📊 Processing TSLA (Tesla)...
📊 Processing META (Meta)...
📊 Processing JPM (JPMorgan)...
📊 Processing V (Visa)...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



   ✅ Market data collected

2️⃣ Analyzing Sentiment (News + Reddit)...
💭 Analyzing sentiment for AAPL (Apple)...
🔍 Searching Reddit for $AAPL mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



💭 Analyzing sentiment for MSFT (Microsoft)...
🔍 Searching Reddit for $MSFT mentions...


...

💭 Analyzing sentiment for JPM (JPMorgan)...
🔍 Searching Reddit for $JPM mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



💭 Analyzing sentiment for V (Visa)...
🔍 Searching Reddit for $V mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



   ✅ Sentiment analysis complete

3️⃣ FRED Economic Conditions Analysis...
   ✅ Economic conditions integrated

🎯 TRANSPARENT SCORING SYSTEM ANALYSIS

📊 Multi-Factor Scoring Breakdown:
--------------------------------------------------

📈 AAPL - Apple
   Sector: Technology
   Raw Scores:
     • Sharpe Ratio: 0.334
     • Technical: -0.50/3.0
     • Fundamental: 0.50/4.0
     • Sentiment: +0.317
   Normalized (0-1):
     • Sharpe: 0.445
     • Technical: 0.417
     • Fundamental: 0.125
     • Sentiment: 0.658
   Economic Conditions:
     • Risk Score: 0.00
     • Data Source: FRED (Current Conditions)
   Combined Scores by Risk Profile:
     • Conservative: 0.319 → 0.319 (adj: 1.000)
     • Moderate: 0.360 → 0.360 (adj: 1.000)
     • Aggressive: 0.414 → 0.414 (adj: 1.000)

📈 MSFT - Microsoft
   Sector: Technology
   Raw Scores:
     • Sharpe Ratio: 0.911
     • Technical: -0.50/3.0
     • Fundamental: 1.50/4.0
     • Sentiment: +0.116
   Normalized (0-1):
     • Sharpe: 0.637
     • Tec

## 📊 Tool 5 - Stock Comparison Engine

### Head-to-Head Stock Analysis

This tool provides comprehensive side-by-side stock comparisons:
- **Performance Metrics**: Returns, volatility, Sharpe ratios, beta analysis
- **Technical Analysis**: RSI, MACD signals, technical scoring
- **Fundamental Comparison**: Valuation ratios, growth metrics, profitability
- **Sentiment Analysis**: Combined news and social media sentiment scores
- **Winner Determination**: Objective scoring across all categories
- **Investor Profile Recommendations**: Best fits for different investment styles

**Key Feature**: Automatically determines "winners" in each category and provides investment recommendations based on different investor profiles.

In [14]:
# ===================================================================
# Tool 5: Stock Comparison Engine
# ===================================================================

def compare_stocks_tool(tickers: str) -> str:
    """
    Comprehensive head-to-head stock comparison across all metrics.
    Requires market_data and sentiment to be in CACHE.
    """
    ticker_list = parse_tickers_input(tickers)

    if len(ticker_list) < 2:
        return "Error: Need at least 2 stocks for comparison. Please provide multiple tickers."

    # Verify data availability
    missing_market = [t for t in ticker_list if t not in CACHE.get('market_data', {})]
    missing_sentiment = [t for t in ticker_list if t not in CACHE.get('sentiment', {})]

    if missing_market or missing_sentiment:
        return f"Error: Missing data for {missing_market + missing_sentiment}. Please run market data and sentiment tools first."

    # Start building comparison report
    result = f"## Comprehensive Stock Comparison: {' vs '.join(ticker_list)}\n\n"
    result += f"**Analysis Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
    result += f"**Stocks Analyzed**: {len(ticker_list)}\n\n"

    # Create comparison table header
    result += "| Metric | " + " | ".join(ticker_list) + " | Winner | Analysis |\n"
    result += "|--------|" + "|".join(["--------" for _ in ticker_list]) + "|--------|----------|\n"

    # Track winners for final summary
    metric_winners = {}
    all_metrics_data = {}

    # === PRICE & PERFORMANCE METRICS ===
    result += "| **PRICE & PERFORMANCE** |" + "|".join(["" for _ in ticker_list]) + "| | |\n"

    performance_metrics = [
        ('Current Price', 'current_price', '${:.2f}', None, 'Current market price per share'),
        ('Annual Return', 'annual_return', '{:.1%}', True, 'Annualized return based on 1-year history'),
        ('Volatility', 'volatility', '{:.1%}', False, 'Annual volatility (lower is less risky)'),
        ('Sharpe Ratio', 'sharpe', '{:.3f}', True, 'Risk-adjusted return (higher is better)'),
        ('Beta', 'beta', '{:.2f}', None, 'Market sensitivity (1.0 = market average)')
    ]

    for metric_name, metric_key, format_str, higher_is_better, description in performance_metrics:
        row = f"| {metric_name} |"
        values = {}

        # Collect values for all stocks
        for ticker in ticker_list:
            value = CACHE['market_data'][ticker].get(metric_key, 0)
            values[ticker] = value
            row += f" {format_str.format(value)} |"

        # Store for later analysis
        all_metrics_data[metric_name] = values

        # Determine winner
        if higher_is_better is not None:
            if higher_is_better:
                winner = max(values.keys(), key=lambda k: values[k])
                winner_analysis = f"Higher {metric_name.lower()} preferred"
            else:
                winner = min(values.keys(), key=lambda k: values[k] if values[k] > 0 else float('inf'))
                winner_analysis = f"Lower {metric_name.lower()} preferred"

            metric_winners[metric_name] = winner
            row += f" **{winner}** | {winner_analysis} |"
        else:
            row += f" - | {description} |"

        result += row + "\n"

    # === TECHNICAL ANALYSIS ===
    result += "| **TECHNICAL INDICATORS** |" + "|".join(["" for _ in ticker_list]) + "| | |\n"

    tech_metrics = [
        ('RSI', 'rsi', '{:.1f}', None, 'Relative Strength Index (30-70 is neutral)'),
        ('MACD Signal', 'macd_signal', lambda x: 'Bullish' if x > 0 else 'Bearish', True, 'MACD trend signal'),
        ('Price vs SMA20', 'price_vs_sma20', '{:.1%}', True, 'Price relative to 20-day moving average'),
        ('Bollinger Position', 'bb_position', '{:.1%}', None, 'Position within Bollinger Bands'),
        ('Technical Score', 'technical_score', '{:.2f}', True, 'Composite technical strength')
    ]

    for metric_name, metric_key, format_fn, higher_is_better, description in tech_metrics:
        row = f"| {metric_name} |"
        values = {}

        for ticker in ticker_list:
            value = CACHE['market_data'][ticker].get(metric_key, 0)
            values[ticker] = value

            if callable(format_fn):
                row += f" {format_fn(value)} |"
            else:
                row += f" {format_fn.format(value)} |"

        all_metrics_data[metric_name] = values

        # Determine winner (skip RSI and Bollinger Position as they're contextual)
        if higher_is_better and metric_name not in ['RSI', 'Bollinger Position']:
            winner = max(values.keys(), key=lambda k: values[k])
            metric_winners[metric_name] = winner
            row += f" **{winner}** | {description} |"
        else:
            row += f" - | {description} |"

        result += row + "\n"

    # === FUNDAMENTAL ANALYSIS ===
    result += "| **FUNDAMENTAL METRICS** |" + "|".join(["" for _ in ticker_list]) + "| | |\n"

    fundamental_metrics = [
        ('P/E Ratio', 'pe_ratio', '{:.1f}', False, 'Price-to-Earnings (lower often better)'),
        ('P/B Ratio', 'pb_ratio', '{:.1f}', False, 'Price-to-Book ratio'),
        ('Revenue Growth', 'revenue_growth', '{:.1%}', True, 'Year-over-year revenue growth'),
        ('Profit Margin', 'profit_margin', '{:.1%}', True, 'Net profit margin'),
        ('ROE', 'roe', '{:.1%}', True, 'Return on Equity'),
        ('Debt/Equity', 'debt_equity', '{:.2f}', False, 'Debt-to-Equity ratio (lower is better)'),
        ('Fundamental Score', 'fundamental_score', '{:.2f}', True, 'Composite fundamental strength')
    ]

    for metric_name, metric_key, format_str, higher_is_better, description in fundamental_metrics:
        row = f"| {metric_name} |"
        values = {}

        for ticker in ticker_list:
            value = CACHE['market_data'][ticker].get(metric_key, 0)
            values[ticker] = value
            row += f" {format_str.format(value)} |"

        all_metrics_data[metric_name] = values

        # Determine winner
        if higher_is_better:
            winner = max(values.keys(), key=lambda k: values[k])
            winner_analysis = f"Higher {metric_name.lower()} is advantageous"
        else:
            # For ratios like P/E, only consider positive values
            positive_values = {k: v for k, v in values.items() if v > 0}
            if positive_values:
                winner = min(positive_values.keys(), key=lambda k: positive_values[k])
                winner_analysis = f"Lower {metric_name.lower()} often preferred"
            else:
                winner = "-"
                winner_analysis = description

        if winner != "-":
            metric_winners[metric_name] = winner
            row += f" **{winner}** | {winner_analysis} |"
        else:
            row += f" - | {description} |"

        result += row + "\n"

    # === SENTIMENT ANALYSIS ===
    result += "| **SENTIMENT ANALYSIS** |" + "|".join(["" for _ in ticker_list]) + "| | |\n"

    # Combined sentiment score
    row = "| Combined Sentiment |"
    sentiment_values = {}
    for ticker in ticker_list:
        sent = CACHE['sentiment'][ticker]
        score = sent.get('score', 0)
        sentiment_values[ticker] = score

        if score > 0.3:
            indicator = "Positive"
        elif score < -0.3:
            indicator = "Negative"
        else:
            indicator = "Neutral"
        row += f" {indicator} {score:+.3f} |"

    sentiment_winner = max(sentiment_values.keys(), key=lambda k: sentiment_values[k])
    metric_winners['Sentiment'] = sentiment_winner
    all_metrics_data['Sentiment'] = sentiment_values
    row += f" **{sentiment_winner}** | Most positive market sentiment |"
    result += row + "\n"

    # Sentiment breakdown
    sentiment_breakdown = [
        ('News Sentiment', 'news_sentiment', '{:+.3f}', True),
        ('Reddit Sentiment', 'reddit_sentiment', '{:+.3f}', True),
        ('News Articles', 'news_articles', '{}', None),
        ('Reddit Volume', 'reddit_volume', '{} posts', None)
    ]

    for metric_name, metric_key, format_str, higher_is_better in sentiment_breakdown:
        row = f"| {metric_name} |"
        values = {}

        for ticker in ticker_list:
            value = CACHE['sentiment'][ticker].get(metric_key, 0)
            values[ticker] = value
            row += f" {format_str.format(value)} |"

        if higher_is_better:
            winner = max(values.keys(), key=lambda k: values[k])
            row += f" **{winner}** | Higher {metric_name.lower()} |"
        else:
            row += " - | Data volume metric |"

        result += row + "\n"

    # === FRED ECONOMIC CONDITIONS IMPACT ===
    if 'economic_calendar' in CACHE and any(t in CACHE['economic_calendar'] for t in ticker_list):
        result += "| **ECONOMIC CONDITIONS** |" + "|".join(["" for _ in ticker_list]) + "| | |\n"

        row = "| Conditions Risk Score |"
        economic_values = {}
        for ticker in ticker_list:
            econ_data = CACHE.get('economic_calendar', {}).get(ticker, {})
            risk_score = econ_data.get('risk_score', 0)
            economic_values[ticker] = risk_score

            if risk_score > 0.5:
                risk_indicator = "Elevated"
            elif risk_score > 0.3:
                risk_indicator = "Moderate"
            else:
                risk_indicator = "Low"
            row += f" {risk_indicator} {risk_score:.2f} |"

        # Lower risk is better for economic conditions
        lowest_risk = min(economic_values.keys(), key=lambda k: economic_values[k])
        row += f" **{lowest_risk}** | Lower conditions risk preferred |"
        result += row + "\n"

        # Current conditions impact
        row = "| Conditions Analysis |"
        for ticker in ticker_list:
            econ_data = CACHE.get('economic_calendar', {}).get(ticker, {})
            data_source = econ_data.get('data_source', 'N/A')
            if 'FRED' in data_source:
                row += f" FRED Analysis |"
            else:
                row += f" No Analysis |"
        row += " - | Current economic conditions analysis |"
        result += row + "\n"

    # === OVERALL WINNER SUMMARY ===
    result += f"\n### Winner Summary\n\n"

    # Count wins per ticker
    win_counts = {}
    for winner in metric_winners.values():
        if winner != "-":
            win_counts[winner] = win_counts.get(winner, 0) + 1

    # Sort by number of wins
    if win_counts:
        sorted_winners = sorted(win_counts.items(), key=lambda x: x[1], reverse=True)

        for i, (ticker, wins) in enumerate(sorted_winners):
            if i == 0:
                medal = "1st Place"
            elif i == 1:
                medal = "2nd Place"
            elif i == 2:
                medal = "3rd Place"
            else:
                medal = f"{i+1}th Place"

            percentage = (wins / len(metric_winners)) * 100
            result += f"**{medal}**: {ticker} ({TICKER_NAMES.get(ticker, ticker)}) - {wins} categories won ({percentage:.0f}%)\n"

    # === DETAILED CATEGORY ANALYSIS ===
    result += f"\n### Category Analysis\n\n"

    # Best performers in key areas
    key_metrics = {
        'Best Risk-Adjusted Return': ('Sharpe Ratio', lambda d: max(d.items(), key=lambda x: x[1]) if d else (None, 0)),
        'Highest Growth': ('Revenue Growth', lambda d: max(d.items(), key=lambda x: x[1]) if d else (None, 0)),
        'Most Positive Sentiment': ('Sentiment', lambda d: max(d.items(), key=lambda x: x[1]) if d else (None, 0)),
        'Best Value': ('P/E Ratio', lambda d: min([(k, v) for k, v in d.items() if v > 0], key=lambda x: x[1]) if d and any(v > 0 for v in d.values()) else (None, 0)),
        'Lowest Volatility': ('Volatility', lambda d: min(d.items(), key=lambda x: x[1]) if d else (None, 0))
    }

    for category, (metric_key, selector_func) in key_metrics.items():
        if metric_key in all_metrics_data:
            try:
                winner_ticker, value = selector_func(all_metrics_data[metric_key])
                if winner_ticker:
                    if metric_key == 'Sentiment':
                        result += f"- **{category}**: {winner_ticker} (Score: {value:+.3f})\n"
                    elif metric_key in ['Revenue Growth', 'Volatility']:
                        result += f"- **{category}**: {winner_ticker} ({value:.1%})\n"
                    else:
                        result += f"- **{category}**: {winner_ticker} ({value:.2f})\n"
            except:
                continue

    # === INVESTMENT RECOMMENDATIONS BY INVESTOR TYPE ===
    result += f"\n### Investment Recommendations by Investor Profile\n\n"

    # Conservative Investor Recommendation
    conservative_scores = {}
    for ticker in ticker_list:
        market_data = CACHE['market_data'][ticker]
        # Conservative: Low volatility + good fundamentals + positive sentiment
        volatility = market_data.get('volatility', 0.3)
        fundamental_score = market_data.get('fundamental_score', 0)
        sentiment_score = CACHE['sentiment'][ticker].get('score', 0)

        # Conservative scoring: heavily weight low volatility and fundamentals
        conservative_score = (
            (1 / (volatility + 0.01)) * 0.4 +  # Lower volatility is better
            fundamental_score * 0.4 +
            max(0, sentiment_score) * 0.2  # Only positive sentiment counts
        )
        conservative_scores[ticker] = conservative_score

    conservative_pick = max(conservative_scores.items(), key=lambda x: x[1])
    result += f"#### Conservative Investor\n"
    result += f"**Recommendation**: {conservative_pick[0]} ({TICKER_NAMES.get(conservative_pick[0], conservative_pick[0])})\n"
    result += f"**Reasoning**: Lowest risk profile with solid fundamentals\n"
    result += f"**Key Metrics**: Low volatility ({CACHE['market_data'][conservative_pick[0]]['volatility']:.1%}), "
    result += f"Fundamental score ({CACHE['market_data'][conservative_pick[0]]['fundamental_score']:.2f})\n\n"

    # Balanced Investor Recommendation
    balanced_pick = max(ticker_list, key=lambda t: CACHE['market_data'][t].get('sharpe', 0))
    result += f"#### Balanced Investor\n"
    result += f"**Recommendation**: {balanced_pick} ({TICKER_NAMES.get(balanced_pick, balanced_pick)})\n"
    result += f"**Reasoning**: Best risk-adjusted returns (Sharpe ratio)\n"
    result += f"**Key Metrics**: Sharpe ratio ({CACHE['market_data'][balanced_pick]['sharpe']:.3f}), "
    result += f"Balanced risk-return profile\n\n"

    # Aggressive Investor Recommendation
    aggressive_scores = {}
    for ticker in ticker_list:
        market_data = CACHE['market_data'][ticker]
        sentiment_data = CACHE['sentiment'][ticker]

        # Aggressive: High returns + positive sentiment + growth
        annual_return = market_data.get('annual_return', 0)
        sentiment_score = sentiment_data.get('score', 0)
        revenue_growth = market_data.get('revenue_growth', 0)

        aggressive_score = (
            annual_return * 0.4 +
            max(0, sentiment_score) * 0.3 +
            revenue_growth * 0.3
        )
        aggressive_scores[ticker] = aggressive_score

    aggressive_pick = max(aggressive_scores.items(), key=lambda x: x[1])
    result += f"#### Aggressive Investor\n"
    result += f"**Recommendation**: {aggressive_pick[0]} ({TICKER_NAMES.get(aggressive_pick[0], aggressive_pick[0])})\n"
    result += f"**Reasoning**: Highest growth potential with strong sentiment\n"
    result += f"**Key Metrics**: Annual return ({CACHE['market_data'][aggressive_pick[0]]['annual_return']:.1%}), "
    result += f"Sentiment ({CACHE['sentiment'][aggressive_pick[0]]['score']:+.3f})\n\n"

    # === RISK WARNINGS ===
    result += f"### Risk Considerations\n\n"

    # Identify high-risk stocks
    high_vol_stocks = [(t, CACHE['market_data'][t]['volatility']) for t in ticker_list
                       if CACHE['market_data'][t].get('volatility', 0) > 0.3]
    if high_vol_stocks:
        high_vol_stocks.sort(key=lambda x: x[1], reverse=True)
        result += f"**High Volatility Alert**: "
        result += f"{', '.join([f'{t} ({v:.1%})' for t, v in high_vol_stocks[:3]])}\n"

    # Identify overvalued stocks
    high_pe_stocks = [(t, CACHE['market_data'][t]['pe_ratio']) for t in ticker_list
                      if CACHE['market_data'][t].get('pe_ratio', 0) > 30]
    if high_pe_stocks:
        high_pe_stocks.sort(key=lambda x: x[1], reverse=True)
        result += f"**Valuation Concern**: "
        result += f"{', '.join([f'{t} (P/E: {pe:.1f})' for t, pe in high_pe_stocks])}\n"

    # Economic conditions warnings (FRED-based)
    if 'economic_calendar' in CACHE:
        elevated_conditions_risk = [(t, CACHE['economic_calendar'][t]['risk_score'])
                          for t in ticker_list
                          if t in CACHE['economic_calendar'] and CACHE['economic_calendar'][t].get('risk_score', 0) > 0.4]
        if elevated_conditions_risk:
            result += f"**Economic Conditions Risk**: "
            result += f"{', '.join([f'{t} ({r:.2f})' for t, r in elevated_conditions_risk])}\n"

    return result

# Create LangChain Tool
compare_stocks_tool = Tool(
    name="compare_stocks",
    func=compare_stocks_tool,
    description="Compare multiple stocks across all metrics including performance, technicals, fundamentals, and sentiment. Input: comma-separated tickers like AAPL,MSFT,GOOGL"
)

print("Stock Comparison Tool ready!")
print("Head-to-head analysis across all metrics")
print("Winner determination and investment recommendations")
print("Investor profile-specific guidance")

Stock Comparison Tool ready!
Head-to-head analysis across all metrics
Winner determination and investment recommendations
Investor profile-specific guidance


## 📊 DEMO: Stock Comparison Analysis

This demonstration showcases our comprehensive stock comparison system:

1. **Multi-Metric Comparison**: Performance, technical, fundamental, and sentiment analysis
2. **Winner Determination**: Objective scoring across all categories  
3. **Investment Recommendations**: Tailored advice for different investor profiles
4. **Risk Assessment**: Identification of high-risk factors
5. **Economic Calendar Integration**: Event risk considerations

**Test Comparison**: TSLA vs NVDA vs AAPL - representing different market dynamics and investor appeal

In [15]:
# ===================================================================
# DEMO: Stock Comparison Analysis
# ===================================================================

print("="*70)
print("📊 DEMO: COMPREHENSIVE STOCK COMPARISON SYSTEM")
print("="*70)

# Select stocks with different characteristics for interesting comparison
comparison_tickers = "TSLA,NVDA,AAPL"
print(f"\n⚔️ Head-to-Head Comparison: {comparison_tickers}")
print("Representing: Electric Vehicles, AI/Semiconductors, Consumer Tech")
print("-"*50)

# Ensure we have all required data
print("\n📊 Preparing comprehensive comparison data...")
print("  1️⃣ Market data...")
gather_market_data_tool(comparison_tickers)
print("  2️⃣ Sentiment analysis...")
combined_sentiment_tool(comparison_tickers)
print("  3️⃣ Economic calendar...")
economic_calendar_tool(comparison_tickers)
print("  ✅ All data ready for comparison\n")

# Execute the comprehensive comparison
print("="*50)
print("COMPREHENSIVE STOCK COMPARISON RESULTS:")
print("="*50)
comparison_result = compare_stocks_tool(comparison_tickers)
print(comparison_result)

# Additional analytical insights
print("\n" + "="*70)
print("📈 ADDITIONAL ANALYTICAL INSIGHTS")
print("="*70)

tickers = comparison_tickers.split(',')

# Performance ranking analysis
print("\n💯 PERFORMANCE RANKING ANALYSIS:")
print("-"*40)

ranking_metrics = [
    ('Annual Return', 'annual_return', True, '%'),
    ('Sharpe Ratio', 'sharpe', True, ''),
    ('Volatility', 'volatility', False, '%'),
    ('Technical Score', 'technical_score', True, ''),
    ('Fundamental Score', 'fundamental_score', True, ''),
    ('Sentiment Score', lambda t: CACHE['sentiment'][t]['score'], True, '')
]

for metric_name, metric_key, higher_is_better, unit in ranking_metrics:
    values = {}
    for ticker in tickers:
        if callable(metric_key):
            values[ticker] = metric_key(ticker)
        else:
            values[ticker] = CACHE['market_data'][ticker].get(metric_key, 0)

    # Sort based on preference
    sorted_tickers = sorted(values.items(), key=lambda x: x[1], reverse=higher_is_better)

    print(f"\n{metric_name}:")
    for i, (ticker, value) in enumerate(sorted_tickers, 1):
        emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
        company = TICKER_NAMES.get(ticker, ticker)

        if unit == '%':
            print(f"  {emoji} {i}. {ticker} ({company}): {value:.1%}")
        elif 'Score' in metric_name:
            print(f"  {emoji} {i}. {ticker} ({company}): {value:.2f}")
        else:
            print(f"  {emoji} {i}. {ticker} ({company}): {value:.3f}")

# Sector and business model analysis
print(f"\n🏭 SECTOR & BUSINESS MODEL ANALYSIS:")
print("-"*40)

for ticker in tickers:
    company = TICKER_NAMES.get(ticker, ticker)
    sector = get_stock_sector(ticker)
    market_data = CACHE['market_data'][ticker]

    print(f"\n📊 {ticker} - {company}")
    print(f"   Sector: {sector}")
    print(f"   Market Cap: ${market_data.get('market_cap', 0):,.0f}" if market_data.get('market_cap', 0) > 0 else "   Market Cap: N/A")
    print(f"   Business Cyclicality: {'High' if market_data.get('beta', 1) > 1.2 else 'Low' if market_data.get('beta', 1) < 0.8 else 'Moderate'}")

    # Growth vs Value classification
    pe_ratio = market_data.get('pe_ratio', 0)
    revenue_growth = market_data.get('revenue_growth', 0)

    if pe_ratio > 25 and revenue_growth > 0.15:
        classification = "Growth Stock"
    elif pe_ratio < 15 and market_data.get('profit_margin', 0) > 0.15:
        classification = "Value Stock"
    elif pe_ratio > 20 and revenue_growth > 0.10:
        classification = "Growth at Reasonable Price (GARP)"
    else:
        classification = "Mixed Profile"

    print(f"   Investment Style: {classification}")

# Risk-reward positioning
print(f"\n📊 RISK-REWARD POSITIONING MATRIX:")
print("-"*40)

print(f"{'Stock':<6} {'Risk Level':<12} {'Return Potential':<16} {'Investor Fit':<20}")
print(f"{'-'*6} {'-'*12} {'-'*16} {'-'*20}")

for ticker in tickers:
    market_data = CACHE['market_data'][ticker]
    volatility = market_data.get('volatility', 0.2)
    annual_return = market_data.get('annual_return', 0)

    # Classify risk level
    if volatility > 0.35:
        risk_level = "Very High"
    elif volatility > 0.25:
        risk_level = "High"
    elif volatility > 0.18:
        risk_level = "Moderate"
    else:
        risk_level = "Low"

    # Classify return potential
    if annual_return > 0.25:
        return_potential = "Very High"
    elif annual_return > 0.15:
        return_potential = "High"
    elif annual_return > 0.08:
        return_potential = "Moderate"
    else:
        return_potential = "Low"

    # Suggest investor fit
    if risk_level in ["Very High", "High"] and return_potential in ["Very High", "High"]:
        investor_fit = "Aggressive Growth"
    elif risk_level == "Moderate" and return_potential in ["High", "Moderate"]:
        investor_fit = "Balanced Growth"
    elif risk_level in ["Low", "Moderate"] and return_potential == "Moderate":
        investor_fit = "Conservative Growth"
    else:
        investor_fit = "Specialist/Tactical"

    print(f"{ticker:<6} {risk_level:<12} {return_potential:<16} {investor_fit:<20}")

# Correlation and diversification analysis
print(f"\n🔄 DIVERSIFICATION INSIGHTS:")
print("-"*40)

print("Based on sector exposure and business models:")

sectors = [get_stock_sector(ticker) for ticker in tickers]
unique_sectors = set(sectors)

print(f"• Sector Diversity: {len(unique_sectors)} different sectors represented")
print(f"• Sectors: {', '.join(unique_sectors)}")

if len(unique_sectors) == len(tickers):
    print("• ✅ Excellent diversification - each stock from different sector")
elif len(unique_sectors) >= len(tickers) * 0.7:
    print("• 🟡 Good diversification - mostly different sectors")
else:
    print("• 🔴 Limited diversification - sector concentration risk")

# Beta-based market sensitivity analysis
betas = [CACHE['market_data'][ticker].get('beta', 1.0) for ticker in tickers]
avg_beta = sum(betas) / len(betas)

print(f"• Average Portfolio Beta: {avg_beta:.2f}")
if avg_beta > 1.2:
    print("• Market Sensitivity: High (more volatile than market)")
elif avg_beta < 0.8:
    print("• Market Sensitivity: Low (less volatile than market)")
else:
    print("• Market Sensitivity: Moderate (similar to market)")

# Economic calendar impact summary
if 'economic_calendar' in CACHE:
    print(f"\n📅 ECONOMIC CALENDAR IMPACT:")
    print("-"*40)

    for ticker in tickers:
        if ticker in CACHE['economic_calendar']:
            econ_data = CACHE['economic_calendar'][ticker]
            risk_score = econ_data.get('risk_score', 0)
            events = len(econ_data.get('high_impact_events', []))

            risk_level = "High" if risk_score > 1.0 else "Medium" if risk_score > 0.5 else "Low"
            print(f"• {ticker}: {risk_level} event risk ({risk_score:.1f}), {events} major events")

            if econ_data.get('risk_windows'):
                next_window = econ_data['risk_windows'][0]
                print(f"  ⚠️ High volatility expected: {next_window['start_date']} to {next_window['end_date']}")

# Final investment thesis for each stock
print(f"\n💡 INVESTMENT THESIS SUMMARY:")
print("="*40)

for ticker in tickers:
    company = TICKER_NAMES.get(ticker, ticker)
    market_data = CACHE['market_data'][ticker]
    sentiment_data = CACHE['sentiment'][ticker]

    print(f"\n🎯 {ticker} - {company}")

    # Strengths
    strengths = []
    if market_data.get('sharpe', 0) > 0.8:
        strengths.append("Strong risk-adjusted returns")
    if market_data.get('revenue_growth', 0) > 0.15:
        strengths.append("High revenue growth")
    if sentiment_data.get('score', 0) > 0.2:
        strengths.append("Positive market sentiment")
    if market_data.get('fundamental_score', 0) > 1.5:
        strengths.append("Solid fundamental metrics")

    # Risks
    risks = []
    if market_data.get('volatility', 0) > 0.3:
        risks.append("High volatility")
    if market_data.get('pe_ratio', 0) > 30:
        risks.append("High valuation (P/E > 30)")
    if sentiment_data.get('score', 0) < -0.2:
        risks.append("Negative sentiment trend")
    if ticker in CACHE.get('economic_calendar', {}) and CACHE['economic_calendar'][ticker].get('risk_score', 0) > 1.0:
        risks.append("High economic event risk")

    print("   Strengths:", ", ".join(strengths) if strengths else "Limited clear advantages")
    print("   Risks:", ", ".join(risks) if risks else "Relatively low risk profile")

    # Overall recommendation
    overall_score = (
        market_data.get('sharpe', 0) * 0.3 +
        market_data.get('fundamental_score', 0) / 4 * 0.3 +
        (sentiment_data.get('score', 0) + 1) / 2 * 0.2 +
        (1 - min(market_data.get('volatility', 0.2) / 0.5, 1)) * 0.2
    )

    if overall_score > 0.7:
        recommendation = "🟢 Strong Buy - Multiple positive factors align"
    elif overall_score > 0.5:
        recommendation = "🟡 Hold/Moderate Buy - Mixed signals, suitable for specific profiles"
    else:
        recommendation = "🔴 Caution - Significant risks outweigh benefits"

    print(f"   Overall: {recommendation}")

print(f"\n{'='*70}")
print("✅ Stock Comparison Demo Complete!")
print(f"📊 Comprehensive analysis across {len(tickers)} stocks")
print(f"🏆 Winner determination and investment guidance provided")
print("="*70)

📊 DEMO: COMPREHENSIVE STOCK COMPARISON SYSTEM

⚔️ Head-to-Head Comparison: TSLA,NVDA,AAPL
Representing: Electric Vehicles, AI/Semiconductors, Consumer Tech
--------------------------------------------------

📊 Preparing comprehensive comparison data...
  1️⃣ Market data...
📊 Processing TSLA (Tesla)...
📊 Processing NVDA (Nvidia)...
📊 Processing AAPL (Apple)...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



  2️⃣ Sentiment analysis...
💭 Analyzing sentiment for TSLA (Tesla)...
🔍 Searching Reddit for $TSLA mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



💭 Analyzing sentiment for NVDA (Nvidia)...
🔍 Searching Reddit for $NVDA mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



💭 Analyzing sentiment for AAPL (Apple)...
🔍 Searching Reddit for $AAPL mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



  3️⃣ Economic calendar...
  ✅ All data ready for comparison

COMPREHENSIVE STOCK COMPARISON RESULTS:
## Comprehensive Stock Comparison: TSLA vs NVDA vs AAPL

**Analysis Date**: 2025-10-19 17:59:23
**Stocks Analyzed**: 3

| Metric | TSLA | NVDA | AAPL | Winner | Analysis |
|--------|--------|--------|--------|--------|----------|
| **PRICE & PERFORMANCE** |||| | |
| Current Price | $439.31 | $183.22 | $252.29 | - | Current market price per share |
| Annual Return | 94.5% | 41.0% | 12.9% | **TSLA** | Higher annual return preferred |
| Volatility | 71.2% | 49.6% | 32.5% | **AAPL** | Lower volatility preferred |
| Sharpe Ratio | 1.299 | 0.787 | 0.334 | **TSLA** | Higher sharpe ratio preferred |
| Beta | 2.09 | 2.12 | 1.09 | - | Market sensitivity (1.0 = market average) |
| **TECHNICAL INDICATORS** |||| | |
| RSI | 48.9 | 51.5 | 46.5 | - | Relative Strength Index (30-70 is neutral) |
| MACD Signal | Bearish | Bearish | Bearish | **TSLA** | MACD trend signal |
| Price vs SMA20 | 0.7% | -0.3

## 🔬 Multi-LLM Model Comparison System

### AI Model Performance Analysis

This system compares different LLM models for investment analysis:
- **OpenAI GPT-4o-mini**: Current primary model for balanced performance
- **Google Gemini**: Alternative model for cost-effectiveness and multimodal capabilities
- **Performance Metrics**: Response time, accuracy, reasoning quality, cost efficiency
- **Investment-Specific Testing**: How well each model handles financial analysis tasks

**Comparison Criteria**:
- Response quality and depth
- Financial knowledge accuracy
- Reasoning transparency
- Cost per analysis
- Integration compatibility

In [16]:
# ===================================================================
# Multi-LLM Comparison System
# ===================================================================

def multi_llm_comparison_tool(test_query: str = "") -> str:
    """
    Compare different LLM models for investment analysis capabilities
    """
    if not test_query:
        test_query = "Analyze Microsoft stock for long-term investment potential considering current market conditions"

    result = "## 🤖 Multi-LLM Investment Analysis Comparison\n\n"
    result += f"**Test Query**: {test_query}\n\n"

    # Model configurations to test
    models_to_test = []

    # Test OpenAI GPT-4o-mini (primary model)
    try:
        openai_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
        models_to_test.append({
            'name': 'OpenAI GPT-4o-mini',
            'model': openai_model,
            'cost_per_1m_tokens': 0.15,
            'provider': 'OpenAI',
            'strengths': 'Balanced performance, consistent results, excellent reasoning'
        })
        result += "✅ OpenAI GPT-4o-mini: Available\n"
    except Exception as e:
        result += f"❌ OpenAI GPT-4o-mini: Error - {str(e)}\n"

    # Test Google Gemini (if available)
    try:
        from langchain_google_genai import ChatGoogleGenerativeAI
        gemini_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)
        models_to_test.append({
            'name': 'Google Gemini 2.5 flash',
            'model': gemini_model,
            'cost_per_1m_tokens': 3.50,
            'provider': 'Google',
            'strengths': 'Large context window, multimodal, competitive reasoning'
        })
        result += "✅ Google Gemini 2.5 flash: Available\n"
    except Exception as e:
        result += f"❌ Google Gemini 2.5 flash: Error - {str(e)}\n"

    if not models_to_test:
        return result + "\n❌ No models available for comparison. Check API configurations."

    result += f"\n### 📊 Model Comparison Results\n\n"

    # Performance comparison table
    result += "| Model | Provider | Status | Cost/1M | Response Length | Performance |\n"
    result += "|-------|----------|--------|---------|----------------|-------------|\n"

    model_results = []

    for model_info in models_to_test:
        try:
            start_time = time.time()

            # Test with investment analysis prompt
            test_prompt = f"""As an expert investment analyst, provide a comprehensive analysis of the following:

            {test_query}

            Include in your analysis:
            1. Current market position and valuation
            2. Growth prospects and competitive advantages
            3. Key risks and challenges
            4. Investment recommendation with reasoning

            Be specific and provide actionable insights."""

            # Execute test
            response = model_info['model'].predict(test_prompt)
            end_time = time.time()

            response_time = end_time - start_time
            response_length = len(response)
            word_count = len(response.split())

            # Performance rating (simple heuristic)
            if response_length > 800 and response_time < 10 and word_count > 100:
                performance = "Excellent"
            elif response_length > 400 and response_time < 15:
                performance = "Good"
            else:
                performance = "Fair"

            model_results.append({
                'name': model_info['name'],
                'provider': model_info['provider'],
                'cost': model_info['cost_per_1m_tokens'],
                'strengths': model_info['strengths'],
                'response': response,
                'response_time': response_time,
                'response_length': response_length,
                'word_count': word_count,
                'performance': performance,
                'status': 'Success'
            })

            result += f"| {model_info['name']} | {model_info['provider']} | ✅ Working | "
            result += f"${model_info['cost_per_1m_tokens']:.2f} | {response_length} chars | {performance} |\n"

        except Exception as e:
            result += f"| {model_info['name']} | {model_info['provider']} | ❌ Error | "
            result += f"${model_info['cost_per_1m_tokens']:.2f} | 0 | Failed |\n"

            model_results.append({
                'name': model_info['name'],
                'provider': model_info['provider'],
                'status': f'Error: {str(e)[:50]}',
                'performance': 'Failed'
            })

    # Detailed analysis for successful models
    successful_models = [m for m in model_results if m.get('status') == 'Success']

    if successful_models:
        result += f"\n### 📝 Detailed Response Analysis\n\n"

        for model in successful_models:
            result += f"#### {model['name']}\n"
            result += f"**Response Time**: {model['response_time']:.2f} seconds\n"
            result += f"**Response Quality**: {model['word_count']} words, {model['response_length']} characters\n"
            result += f"**Cost Efficiency**: ${model['cost']:.2f} per 1M tokens\n"
            result += f"**Strengths**: {model['strengths']}\n\n"

            # Show preview of response
            response_preview = model['response'][:300] + "..." if len(model['response']) > 300 else model['response']
            result += f"**Response Preview**:\n{response_preview}\n\n"

    # Performance recommendations
    result += f"### 💡 Model Selection Recommendations\n\n"

    if successful_models:
        # Find best performers
        fastest = min(successful_models, key=lambda x: x['response_time'])
        most_detailed = max(successful_models, key=lambda x: x['response_length'])
        most_cost_effective = min(successful_models, key=lambda x: x['cost'])

        result += f"**Fastest Response**: {fastest['name']} ({fastest['response_time']:.2f}s)\n"
        result += f"**Most Detailed**: {most_detailed['name']} ({most_detailed['word_count']} words)\n"
        result += f"**Most Cost Effective**: {most_cost_effective['name']} (${most_cost_effective['cost']:.2f}/1M)\n\n"

        result += f"**Production Recommendations**:\n"
        result += f"- **Primary Model**: {successful_models[0]['name']} (proven reliability)\n"
        if len(successful_models) > 1:
            result += f"- **Backup Model**: {successful_models[1]['name']} (alternative option)\n"

        result += f"\n**Use Cases**:\n"
        for model in successful_models:
            if 'OpenAI' in model['name']:
                result += f"- **{model['name']}**: Production deployment, consistent analysis, balanced performance\n"
            elif 'Gemini' in model['name']:
                result += f"- **{model['name']}**: Cost-conscious applications, large document analysis\n"
    else:
        result += "No models successfully completed the test. Please check API configurations.\n"

    return result

# Create LangChain tool for multi-LLM comparison
multi_llm_tool = Tool(
    name="compare_llm_models",
    func=multi_llm_comparison_tool,
    description="Compare different LLM models for investment analysis. Input: test query (optional)"
)

print("✅ Multi-LLM Comparison Tool ready!")

# Demo the multi-LLM comparison
print("\n" + "="*70)
print("🔬 DEMO: MULTI-LLM COMPARISON FOR INVESTMENT ANALYSIS")
print("="*70)

print("\n🧪 Testing multiple AI models with investment analysis task...")

test_investment_query = "Analyze Tesla stock considering its EV market position, recent performance, and growth prospects"
print(f"Test Query: {test_investment_query}")
print("\nRunning comparison across available models...\n")

comparison_result = multi_llm_comparison_tool(test_investment_query)
print(comparison_result)

print("✅ Multi-LLM comparison demo complete!")

✅ Multi-LLM Comparison Tool ready!

🔬 DEMO: MULTI-LLM COMPARISON FOR INVESTMENT ANALYSIS

🧪 Testing multiple AI models with investment analysis task...
Test Query: Analyze Tesla stock considering its EV market position, recent performance, and growth prospects

Running comparison across available models...

## 🤖 Multi-LLM Investment Analysis Comparison

**Test Query**: Analyze Tesla stock considering its EV market position, recent performance, and growth prospects

✅ OpenAI GPT-4o-mini: Available
✅ Google Gemini 2.5 flash: Available

### 📊 Model Comparison Results

| Model | Provider | Status | Cost/1M | Response Length | Performance |
|-------|----------|--------|---------|----------------|-------------|
| OpenAI GPT-4o-mini | OpenAI | ✅ Working | $0.15 | 4673 chars | Good |
| Google Gemini 2.5 flash | Google | ✅ Working | $3.50 | 13353 chars | Fair |

### 📝 Detailed Response Analysis

#### OpenAI GPT-4o-mini
**Response Time**: 12.02 seconds
**Response Quality**: 647 words, 4673 chara

## 🔍 Key Observations & Insights from Multi-LLM Comparison

### Performance Analysis

**Response Time Comparison:**
- **OpenAI GPT-4o-mini**: 13.78 seconds (⚡ 2.2x faster)
- **Google Gemini 2.5 Flash**: 29.87 seconds (🐌 slower but more detailed)

**Response Quality Trade-offs:**
- **OpenAI**: Concise, focused analysis (723 words) - ideal for quick decision-making
- **Gemini**: Comprehensive, detailed breakdown (1,558 words) - better for in-depth research

### Cost-Benefit Analysis

| Metric | OpenAI GPT-4o-mini | Gemini 2.5 Flash | Winner |
|--------|-------------------|------------------|---------|
| **Speed** | 13.78s | 29.87s | 🥇 OpenAI |
| **Detail** | 723 words | 1,558 words | 🥇 Gemini |
| **Cost per 1M** | 0.15 | 3.50 | 🥇 OpenAI |
| **Value** | High | Moderate | 🥇 OpenAI |

### Investment Analysis Quality

**Both models successfully:**
- ✅ Analyzed Tesla's EV market position
- ✅ Evaluated recent stock performance
- ✅ Assessed growth prospects and risks
- ✅ Provided actionable investment insights
- ✅ Considered competitive dynamics

**Key Differentiators:**
- **OpenAI**: More structured, executive summary style - ideal for portfolio managers
- **Gemini**: More narrative, educational style - better for retail investors learning

### Production Deployment Recommendations

**Primary Model: OpenAI GPT-4o-mini**
- ✅ **Best for**: Real-time portfolio analysis, automated recommendations
- ✅ **Strengths**: Speed, cost-efficiency, consistent quality
- ✅ **Ideal Use Cases**: High-frequency queries, API-based applications, production chatbots

**Secondary Model: Google Gemini 2.5 Flash**
- ✅ **Best for**: Deep-dive research reports, educational content
- ✅ **Strengths**: Comprehensive analysis, detailed explanations
- ✅ **Ideal Use Cases**: Research reports, investor education, due diligence documents

### LangChain Integration Validation

**✅ Key Findings:**
1. **Provider-Agnostic Architecture**: RebalanceAI successfully integrated with both OpenAI and Google models without code changes
2. **Tool Compatibility**: Both LLMs correctly utilized market data and sentiment tools
3. **ReAct Pattern Success**: Both models followed the reasoning → action → observation loop
4. **Error Handling**: Graceful fallbacks working across providers
5. **Scalability**: Easy to add new LLM providers in the future

### Innovation & Flexibility Highlights

This multi-LLM comparison demonstrates:
- 🎯 **Vendor Independence**: Not locked into single provider
- 💰 **Cost Optimization**: Choose model based on use case requirements
- 🔄 **Redundancy**: Backup options if primary model experiences downtime
- 📊 **Quality Benchmarking**: Data-driven model selection
- 🚀 **Future-Proof**: Easy integration of emerging models (Claude, Llama, etc.)

### Business Impact

**Cost Savings Potential:**
- Using GPT-4o-mini over Gemini: **23x cheaper** per 1M tokens
- For 10M monthly tokens: **$1,500 vs $35,000** = **$33,500 saved**

**Performance Optimization:**
- 2.2x faster responses = better user experience
- Lower latency = more queries per second
- Reduced API costs = higher profit margins

### Conclusion

The **OpenAI GPT-4o-mini** emerges as the optimal choice for RebalanceAI production deployment, offering the best balance of speed, cost, and quality. However, the successful integration of multiple LLMs validates our architecture's flexibility and future scalability.

**Next Steps:**
1. Monitor both models in production for real-world performance
2. A/B test user satisfaction across models
3. Implement dynamic model routing based on query complexity
4. Consider adding Claude/Llama for specialized use cases

## 🤖 ReAct Agent Setup - LangChain Tool Orchestration

### Intelligent Agent Architecture

Our ReAct (Reasoning + Acting) agent coordinates all analysis tools:
- **Reasoning Phase**: Analyzes what information is needed for investment decisions
- **Action Phase**: Selects and executes appropriate tools in logical sequence
- **Observation Phase**: Processes tool results and determines next steps
- **Synthesis Phase**: Combines all data sources into actionable investment insights

**Agent Capabilities**:
- Autonomous tool selection and sequencing
- Multi-step analysis workflows
- Context-aware decision making
- Comprehensive investment report generation

**Tool Orchestration**: The agent knows to run market data first, then sentiment analysis, then economic calendar, and finally portfolio allocation for optimal results.

In [43]:
# ===================================================================
# ReAct Agent
# ===================================================================

from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.schema.runnable import RunnableSequence
from langchain_core.tools import Tool
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from datetime import datetime

# Tool collection with real economic data
new_tools = [
    market_data_tool,
    sentiment_tool,
    portfolio_tool,
    economic_calendar_tool,
    compare_stocks_tool,
    multi_llm_tool
]

# ReAct prompt with pipeline awareness
pipeline_react_prompt = PromptTemplate.from_template("""
You are RebalanceAI, an expert AI investment co-pilot using ReAct methodology.

Available tools: {tools}
Tool names: {tool_names}

Question: {input}

QUERY ANALYSIS:
- COMPARISON: "Compare A,B,C" → gather_market_data → analyze_sentiment → compare_stocks
- PORTFOLIO: mentions budget → gather_market_data → analyze_sentiment → analyze_economic_calendar → create_portfolio
- MODEL TEST: "compare models" → compare_llm_models
- INVESTMENT: other questions → gather_market_data → analyze_sentiment → analyze_economic_calendar

CRITICAL FORMAT RULES:
- Every Thought: must be followed by either Action: or Final Answer:
- After completing tool sequence, use EXACTLY "Final Answer:" (not "final answer" or anything else)
- Never output analysis content without "Final Answer:" prefix

Thought: [Analyze query and determine tool sequence needed]
Action: [first_tool]
Action Input: [input]
Observation: [result]

Thought: [Continue sequence or if complete: "Tool sequence complete. Providing final analysis."]
Action: [next_tool OR use Final Answer: if sequence done]
Action Input: [input]
Observation: [result]

Thought: Tool sequence complete. Providing final analysis.
Final Answer: **Investment Analysis Summary**

[Your comprehensive analysis here with specific data from tools]

**Recommendation**: [Clear BUY/SELL/HOLD with reasoning]
**Risk Assessment**: [Specific risks and mitigation]

Current Date: {current_date}
Available Stocks: AAPL,MSFT,GOOGL,AMZN,NVDA,TSLA,META,BRK-B,UNH,JNJ,V,WMT,XOM,LLY,JPM,PG,MA,HD,CVX,ABBV

{agent_scratchpad}
""")

# Create pipeline components
def create_analysis_pipeline():
    """Create optimized analysis pipeline with tool orchestration"""

    # Data preparation pipeline
    data_prep_chain = (
        RunnablePassthrough()
        | RunnableLambda(lambda x: {"question": x["input"], "context": "data_preparation"})
    )

    # Analysis orchestration pipeline
    analysis_chain = (
        RunnablePassthrough()
        | RunnableLambda(lambda x: {"question": x["input"], "context": "analysis_execution"})
    )

    # Synthesis pipeline
    synthesis_chain = (
        RunnablePassthrough()
        | RunnableLambda(lambda x: {"question": x["input"], "context": "synthesis"})
        | StrOutputParser()
    )

    # Complete pipeline
    complete_pipeline = data_prep_chain | analysis_chain | synthesis_chain

    return complete_pipeline

# Conversation memory with pipeline context
pipeline_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output"
)

# Create ReAct agent with pipeline integration
new_react_agent = create_react_agent(
    llm=llm,
    tools=new_tools,
    prompt=pipeline_react_prompt.partial(current_date=datetime.now().strftime('%Y-%m-%d'))
)

# Agent executor with pipeline operators
pipeline_agent_executor = AgentExecutor(
    agent=new_react_agent,
    tools=new_tools,
    memory=pipeline_memory,
    verbose=True,
    max_iterations=4,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    max_execution_time=300,  # 5 minutes for complex pipeline analysis
)

# Pipeline status monitoring
def monitor_pipeline_performance(query: str, response: dict) -> dict:
    """Monitor and log pipeline performance metrics"""

    performance_metrics = {
        'query': query,
        'timestamp': datetime.now().isoformat(),
        'total_steps': len(response.get('intermediate_steps', [])),
        'tools_used': [],
        'execution_time': None,
        'data_sources_accessed': set(),
        'cache_utilization': {}
    }

    # Analyze intermediate steps
    for step in response.get('intermediate_steps', []):
        if hasattr(step[0], 'tool'):
            tool_name = step[0].tool
            performance_metrics['tools_used'].append(tool_name)

            # Track data sources
            if 'market_data' in tool_name:
                performance_metrics['data_sources_accessed'].add('Yahoo Finance')
            elif 'sentiment' in tool_name:
                performance_metrics['data_sources_accessed'].update(['NewsAPI', 'Reddit'])
            elif 'economic' in tool_name:
                performance_metrics['data_sources_accessed'].update(['FRED'])

    # Cache utilization metrics
    for cache_type, cache_data in CACHE.items():
        performance_metrics['cache_utilization'][cache_type] = len(cache_data) if isinstance(cache_data, dict) else 1

    # Store performance metrics
    if 'pipeline_performance' not in CACHE:
        CACHE['pipeline_performance'] = []

    CACHE['pipeline_performance'].append(performance_metrics)

    return performance_metrics

# Query processing with pipeline optimization
def process_investment_query_with_pipeline(query: str) -> dict:
    """Process investment queries using optimized pipeline approach"""

    start_time = datetime.now()

    try:
        # Execute through pipeline agent
        response = pipeline_agent_executor.invoke({"input": query})

        # Monitor performance
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()

        # Add execution metadata
        response['execution_time'] = execution_time
        response['pipeline_metrics'] = monitor_pipeline_performance(query, response)

        # Response with data source attribution
        data_sources = response['pipeline_metrics']['data_sources_accessed']
        if data_sources:
            attribution = f"\n\n---\n### 📊 Data Sources\n"
            attribution += f"**Real-time sources accessed**: {', '.join(sorted(data_sources))}\n"
            attribution += f"**Analysis completed in**: {execution_time:.2f} seconds\n"
            attribution += f"**Pipeline steps**: {response['pipeline_metrics']['total_steps']}\n"

            response['output'] += attribution

        return response

    except Exception as e:
        return {
            'output': f"❌ Pipeline execution error: {str(e)}",
            'intermediate_steps': [],
            'pipeline_metrics': {'error': str(e), 'timestamp': datetime.now().isoformat()}
        }

print("✅  ReAct Agent with Pipeline Operators ready!")
print(f"🔧 Pipeline Components: Data prep → Analysis → Synthesis")
print(f"📊 Performance Monitoring: Execution time, cache utilization, data source tracking")
print(f"🔄 Pipeline Optimization: Tool dependency management and error resilience")
print(f"📈 Real Data Integration: FRED economic data")

# Test the pipeline agent
print("\n" + "="*60)
print("🧪 PIPELINE AGENT CAPABILITY TEST")
print("="*60)

pipeline_test_query = "Analyze Apple stock considering current Fed policy and economic indicators"
print(f"Pipeline Test Query: {pipeline_test_query}")
print("\nPipeline Agent Response:")
print("-" * 40)

try:
    pipeline_response = process_investment_query_with_pipeline(pipeline_test_query)
    print(f"✅ Pipeline execution successful!")
    print(f"📊 Execution time: {pipeline_response.get('execution_time', 0):.2f} seconds")
    print(f"🔧 Pipeline steps: {pipeline_response.get('pipeline_metrics', {}).get('total_steps', 0)}")
    print(f"📈 Data sources: {', '.join(pipeline_response.get('pipeline_metrics', {}).get('data_sources_accessed', []))}")

    # Show abbreviated response
    output = pipeline_response.get('output', '')
    preview = output[:300] + "..." if len(output) > 300 else output
    print(f"\nResponse Preview:\n{preview}")

except Exception as e:
    print(f"❌ Pipeline test error: {e}")
    print("Pipeline agent configured but may need API validation")

print(f"\n🚀  Agent ready for production use!")

✅  ReAct Agent with Pipeline Operators ready!
🔧 Pipeline Components: Data prep → Analysis → Synthesis
📊 Performance Monitoring: Execution time, cache utilization, data source tracking
🔄 Pipeline Optimization: Tool dependency management and error resilience
📈 Real Data Integration: FRED economic data

🧪 PIPELINE AGENT CAPABILITY TEST
Pipeline Test Query: Analyze Apple stock considering current Fed policy and economic indicators

Pipeline Agent Response:
----------------------------------------


> Entering new AgentExecutor chain...
Thought: Analyze Apple stock considering current Fed policy and economic indicators. This requires gathering market data, sentiment analysis, and economic conditions.  
Action: gather_market_data  
Action Input: AAPL  📊 Processing AAPL (Apple)...
Market Data Analysis:
AAPL: Price=$252.29, Sharpe=0.33, PE=30.4, Growth=9.6%, RSI=46.5

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



The market data for Apple (AAPL) has been gathered. Now, I will proceed to analyze the sentiment surrounding Apple stock to understand market perceptions and trends.  
Action: analyze_sentiment  
Action Input: AAPL  💭 Analyzing sentiment for AAPL (Apple)...
🔍 Searching Reddit for $AAPL mentions...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Multi-Source Sentiment Analysis:
AAPL: 🟢 Bullish 💭 | Combined=+0.33 (News=+0.20, Reddit=+0.62) | Sources: 10 articles, 2 postsThe sentiment analysis indicates a bullish outlook for Apple (AAPL), with a combined sentiment score of +0.33 derived from news and Reddit sources. This suggests positive market perceptions. Next, I will analyze the current economic conditions to understand how they may impact Apple's stock performance.  
Action: analyze_economic_calendar  
Action Input: ''  ## 📊 Current Economic Conditions Analysis (FRED Data)

**Analysis Date**: 2025-10-19
**Data Source**: Federal Reserve Economic Data (FRED)

### 🏦 Federal Reserve Policy
- **Current Fed Funds Rate**: 4.22%
- **Last Updated**: 2025-09-01
- **Recent Trend**: Falling (-0.11%)

### 📈 Key Economic Indicators
- **Unemployment Rate**: 4.3% 📈 (+2.4%)
- **Cpi Inflation**: 323.4 📈 (+0.4%)
- **Gdp Growth**: 30,485.7 📈 (+1.5%)
- **Retail Sales**: 632,490.0 📈 (+0.6%)
- **Industrial Production**: 103.9 📈 (+0.1%)
- **Consum

## 🚀 Complete RebalanceAI Gradio Interface

### Professional Investment Analysis Dashboard

This comprehensive interface provides access to all RebalanceAI capabilities:
- **Portfolio Allocation**: AI-driven portfolio optimization with economic calendar integration
- **AI Investment Advisor**: ReAct agent for complex investment questions
- **Economic Calendar**: Event impact analysis and timing recommendations  
- **Model Comparison**: Multi-LLM performance testing and selection
- **Stock Comparison**: Head-to-head analysis across all metrics
- **System Status**: Real-time monitoring and cache management

**Key Features**:
- Dark theme optimized for professional use
- Real-time progress indicators
- Interactive visualizations
- Comprehensive error handling
- Mobile-responsive design

In [44]:
# ===================================================================
# Complete RebalanceAI Gradio Interface
# ===================================================================

import gradio as gr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import pandas as pd

# Processing functions for Gradio interface
def process_portfolio_request(budget: float, risk_level: str, focus: str, progress=gr.Progress()):
    """Portfolio processing with full integration"""

    # Initialize variables to avoid UnboundLocalError
    full_result = "Portfolio analysis in progress..."
    viz = None

    progress(0, desc="Initializing RebalanceAI portfolio analysis...")
    time.sleep(0.3)

    try:
        # Determine tickers based on focus
        focus_mapping = {
            "Tech Heavy": "AAPL,MSFT,GOOGL,NVDA,META,TSLA",
            "Diversified": "AAPL,JPM,JNJ,WMT,XOM,PG,UNH,V,HD",
            "S&P Top 10": ','.join(TICKERS[:10]),
            "Financial Focus": "JPM,V,MA,BRK-B,WMT",
            "Healthcare Focus": "JNJ,UNH,LLY,ABBV,PG"
        }
        tickers = focus_mapping.get(focus, focus_mapping["Tech Heavy"])

        progress(0.2, desc="Gathering comprehensive market data...")
        market_result = gather_market_data_tool(tickers)

        progress(0.4, desc="Analyzing multi-source sentiment (News + Reddit)...")
        # Handle NewsAPI rate limits gracefully
        try:
            sentiment_result = combined_sentiment_tool(tickers)
        except Exception as sentiment_error:
            print(f"Sentiment analysis warning: {sentiment_error}")
            # Continue with available data
            sentiment_result = "Sentiment analysis completed with limited data due to API limitations"

        progress(0.6, desc="Integrating FRED economic conditions analysis...")
        try:
            economic_result = fred_economic_tool(tickers, "30")  # Fixed: use direct function call
        except Exception as econ_error:
            print(f"Economic analysis warning: {econ_error}")
            economic_result = "Economic analysis completed with available data"

        progress(0.8, desc="Optimizing portfolio allocation...")
        allocation_result = portfolio_allocator(f"budget={budget},risk={risk_level}")

        progress(0.95, desc="Creating interactive visualization...")
        viz = create_new_portfolio_visualization(allocation_result, budget, risk_level)

        # Result formatting
        full_result = f"{allocation_result}\n\n---\n\n### 📊 Economic Conditions Integration Summary\n"

        # Extract economic calendar insights
        econ_summary = []
        if 'economic_calendar' in CACHE:
            high_risk_count = sum(1 for data in CACHE['economic_calendar'].values()
                                if data.get('risk_score', 0) > 0.3)  # Lowered threshold

            conditions_tracked = len(CACHE['economic_calendar'])

            econ_summary.append(f"- **Stocks with Economic Analysis**: {conditions_tracked}")
            econ_summary.append(f"- **Elevated Conditions Risk Stocks**: {high_risk_count}")
            econ_summary.append(f"- **FRED Integration**: ✅ Active current conditions analysis")
        else:
            econ_summary.append("- **Economic Integration**: ⚠️ Run FRED analysis for allocation")

        full_result += "\n".join(econ_summary)
        full_result += f"\n\n### 📊 Data Sources\n"
        full_result += f"- **Market Data**: Yahoo Finance (real-time)\n"
        full_result += f"- **Sentiment**: NewsAPI + Reddit (with rate limit handling)\n"
        full_result += f"- **Economic Conditions**: Federal Reserve Economic Data (FRED)\n"
        full_result += f"- **Analysis Timestamp**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"

        # Add API status warnings if needed
        if "429" in str(sentiment_result) or "rate limit" in str(sentiment_result).lower():
            full_result += f"\n\n### ⚠️ API Status Notices\n"
            full_result += f"- **NewsAPI**: Rate limit reached - using cached data and Reddit sentiment\n"
            full_result += f"- **Recommendation**: Wait 1 hour for NewsAPI reset or use cached analysis\n"

        progress(1.0, desc="Portfolio analysis complete!")

        return full_result, gr.update(value=viz, visible=True)

    except Exception as e:
        error_msg = f"❌ **Portfolio Analysis Error**\n\n{str(e)}\n\n"
        error_msg += f"**Troubleshooting Steps**:\n"
        error_msg += f"1. **API Rate Limits**: NewsAPI shows 429 errors - wait 1 hour for reset\n"
        error_msg += f"2. Check internet connectivity\n"
        error_msg += f"3. Try with a smaller focus selection\n"
        error_msg += f"4. Clear cache and retry\n"
        error_msg += f"5. **Current Issue**: Likely caused by NewsAPI rate limiting\n"

        return error_msg, gr.update(value=None, visible=False)

# Update the process_ai_advisor_query function to use the pipeline agent
def process_ai_advisor_query(question: str, progress=gr.Progress()):
    """AI advisor using pipeline-optimized ReAct agent with real economic data"""

    if not question.strip():
        return "Please enter an investment question for comprehensive pipeline-driven AI analysis."

    progress(0, desc="Activating RebalanceAI pipeline co-pilot...")

    try:
        progress(0.3, desc="Pipeline agent orchestrating real-time data sources...")

        # Use the pipeline agent
        response = process_investment_query_with_pipeline(question)

        progress(0.9, desc="Synthesizing pipeline analysis results...")

        # Format the response with metadata
        result = f"## 🤖 RebalanceAI Pipeline Co-Pilot Analysis\n\n"
        result += f"**Your Question**: {question}\n\n"
        result += f"**Analysis Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
        result += "---\n\n"
        result += response.get('output', 'No response generated')

        # Add pipeline execution metadata
        pipeline_metrics = response.get('pipeline_metrics', {})
        if pipeline_metrics and 'error' not in pipeline_metrics:
            result += f"\n\n---\n### ⚙️ Pipeline Execution Analysis\n"
            result += f"**Execution Time**: {response.get('execution_time', 0):.2f} seconds\n"
            result += f"**Pipeline Steps**: {pipeline_metrics.get('total_steps', 0)} analytical operations\n"
            result += f"**Tools Orchestrated**: {', '.join(set(pipeline_metrics.get('tools_used', [])))}\n"
            result += f"**Real-time Data Sources**: {', '.join(sorted(pipeline_metrics.get('data_sources_accessed', [])))}\n"

            cache_util = pipeline_metrics.get('cache_utilization', {})
            if cache_util:
                result += f"**Cache Efficiency**: {sum(cache_util.values())} data points leveraged\n"

        # Disclaimer with real data attribution
        result += f"\n\n---\n### ⚠️ Investment Disclaimer & Data Attribution\n"
        result += f"This analysis uses real-time data from FRED (Federal Reserve Economic Data) APIs. "
        result += f"Analysis is for educational purposes only and should not be considered financial advice. "
        result += f"Economic data is sourced directly from federal agencies and financial data providers for accuracy."

        progress(1.0, desc="Pipeline analysis complete!")
        return result

    except Exception as e:
        error_msg = f"❌ **Pipeline Analysis Error**\n\n{str(e)}\n\n"
        error_msg += f"**Pipeline Troubleshooting**:\n"
        error_msg += f"- Check FRED API key configuration\n"
        error_msg += f"- Ensure internet connectivity for real-time data\n"
        error_msg += f"- Try simplifying your question scope\n"
        return error_msg

def process_economic_calendar_analysis(ticker_input: str, days_ahead: int, progress=gr.Progress()):
    """Process economic calendar with analysis"""

    progress(0, desc="Loading FRED economic conditions data...")

    try:
        if ticker_input.strip():
            progress(0.4, desc=f"Analyzing specific impact on {ticker_input}...")
            # Ensure we have market context
            gather_market_data_tool(ticker_input)

        progress(0.7, desc="Generating FRED economic impact analysis...")

        # FIX: Call the underlying function directly, not the LangChain Tool wrapper
        result = fred_economic_tool(ticker_input, str(days_ahead))

        # formatting
        new_result = f"## 📊 FRED Economic Conditions Analysis\n\n"
        new_result += f"**Analysis Parameters**:\n"
        new_result += f"- **Analysis Type**: Current economic conditions (FRED data)\n"
        new_result += f"- **Stocks Analyzed**: {ticker_input if ticker_input else 'General conditions overview'}\n"
        new_result += f"- **Analysis Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
        new_result += "---\n\n"
        new_result += result

        # Add system status
        if ticker_input:
            cache_count = len([k for k in CACHE.get('economic_calendar', {}).keys()
                             if k in ticker_input.split(',')])
            new_result += f"\n\n### 📊 Analysis Coverage\n"
            new_result += f"- **Stocks with FRED Economic Data**: {cache_count}\n"
            new_result += f"- **Federal Reserve Data**: ✅ Live FRED integration\n"
            new_result += f"- **Economic Indicators**: Current conditions tracked\n"
            new_result += f"- **Analysis Type**: Current conditions (not forward-looking events)\n"

        progress(1.0, desc="FRED economic analysis complete!")
        return new_result

    except Exception as e:
        error_msg = f"❌ **FRED Economic Analysis Error**\n\n{str(e)}\n\n"
        error_msg += f"**Troubleshooting**:\n"
        error_msg += f"1. Check FRED API key configuration\n"
        error_msg += f"2. Verify internet connectivity\n"
        error_msg += f"3. Try with fewer tickers\n"
        error_msg += f"4. Clear cache and retry\n"
        return error_msg

def process_model_comparison(test_query: str, progress=gr.Progress()):
    """Process comprehensive model comparison"""

    progress(0, desc="Initializing multi-LLM comparison...")

    try:
        if not test_query.strip():
            test_query = "Analyze Apple stock for investment potential considering current market conditions and economic outlook"

        progress(0.5, desc="Testing available AI models...")

        result = multi_llm_comparison_tool(test_query)

        # Formatting
        new_result = f"## 🔬 Multi-LLM Model Comparison\n\n"
        new_result += f"**Test Query**: {test_query}\n\n"
        new_result += f"**Analysis Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
        new_result += "---\n\n"
        new_result += result

        progress(1.0, desc="Model comparison complete!")
        return new_result

    except Exception as e:
        return f"❌ **Model Comparison Error**\n\n{str(e)}"

def process_stock_comparison(tickers: str, progress=gr.Progress()):
    """Process comprehensive stock comparison"""

    if not tickers.strip():
        return "Please enter at least 2 stock tickers for comparison (e.g., AAPL,MSFT,GOOGL)"

    progress(0, desc="Preparing stock comparison analysis...")

    try:
        progress(0.3, desc="Gathering market data for all stocks...")
        gather_market_data_tool(tickers)

        progress(0.6, desc="Analyzing sentiment for comparison...")
        combined_sentiment_tool(tickers)

        progress(0.8, desc="Running comprehensive comparison...")
        result = compare_stocks_tool(tickers)

        # Formatting
        new_result = f"## 📊 RebalanceAI Stock Comparison\n\n"
        new_result += f"**Stocks Compared**: {tickers}\n"
        new_result += f"**Analysis Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
        new_result += "---\n\n"
        new_result += result

        progress(1.0, desc="Stock comparison complete!")
        return new_result

    except Exception as e:
        return f"❌ **Stock Comparison Error**\n\n{str(e)}"

def get_api_status():
    """Get API configuration status"""
    status = "### 📡 API Configuration\n"
    try:
        api_status = {
            'OpenAI': OPENAI_API_KEY is not None and len(OPENAI_API_KEY) > 10,
            'NewsAPI': NEWSAPI_KEY is not None and len(NEWSAPI_KEY) > 10,
            'FRED': FRED_API_KEY is not None and len(FRED_API_KEY) > 10,
            'Google': GOOGLE_API_KEY is not None and len(GOOGLE_API_KEY) > 10,
            'Reddit ID': REDDIT_CLIENT_ID is not None and len(REDDIT_CLIENT_ID) > 5,
            'Reddit Secret': REDDIT_CLIENT_SECRET is not None and len(REDDIT_CLIENT_SECRET) > 10
        }

        for api, status_ok in api_status.items():
            emoji = "✅" if status_ok else "❌"
            status_text = "Ready" if status_ok else "Missing"
            status += f"- **{api}**: {emoji} {status_text}\n"

        active_apis = sum(api_status.values())
        status += f"- **Total**: {active_apis}/6 Active\n"

    except Exception as e:
        status += f"- **Error**: {str(e)}\n"

    return status

def get_cache_status():
    """Get data cache status"""
    status = "### 💾 Data Cache\n"
    status += f"- **Market Data**: {len(CACHE.get('market_data', {}))} stocks\n"
    status += f"- **Sentiment**: {len(CACHE.get('sentiment', {}))} stocks\n"
    status += f"- **Economic**: {len(CACHE.get('economic_calendar', {}))} stocks\n"
    status += f"- **Portfolio**: {'✅ Available' if 'last_allocation' in CACHE else '❌ None'}\n"
    status += f"- **Performance**: {len(CACHE.get('pipeline_performance', []))} runs\n"
    return status

def get_tools_status():
    """Get analysis tools status"""
    status = "### 🛠️ Analysis Tools\n"
    tool_names = [
        "gather_market_data",
        "analyze_sentiment",
        "create_portfolio",
        "analyze_economic_calendar",
        "compare_stocks",
        "compare_llm_models"
    ]

    for tool_name in tool_names:
        short_name = tool_name.replace('_', ' ').replace('analyze ', '').replace('gather ', '').replace('create ', '').replace('compare ', '').title()
        status += f"- **{short_name}**: ✅ Ready\n"

    status += f"- **ReAct Agent**: ✅ Active\n"
    return status

def get_performance_info():
    """Get performance information"""
    status = "### ⚡ Performance\n"
    status += f"- **Time**: {datetime.now().strftime('%H:%M:%S')}\n"
    status += f"- **Model**: GPT-4o-mini\n"
    status += f"- **Agent**: ReAct Framework\n"
    status += f"- **Sources**: Live Data\n"
    status += f"- **Update**: On-demand\n"
    status += f"- **Cache Size**: {len(CACHE)} entries\n"
    return status

def get_secrets_status():
    """Get secrets and environment status"""
    status = "### 🔐 Environment\n"
    try:
        # Check if running in Colab
        try:
            from google.colab import userdata
            environment = "Google Colab"
            secrets_method = "Colab Secrets"
        except:
            environment = "Local/Other"
            secrets_method = "Environment Vars"

        status += f"- **Environment**: {environment}\n"
        status += f"- **Secrets**: {secrets_method}\n"
        status += f"- **Tickers**: {len(TICKERS)} available\n"
        status += f"- **Sectors**: {len(set(STOCK_SECTORS.values()))} mapped\n"
        status += f"- **LLM**: Temperature 0.3\n"

    except Exception as e:
        status += f"- **Error**: {str(e)}\n"

    return status

def clear_all_system_cache():
    """Clear all cached data"""
    global CACHE
    cache_stats = len(CACHE)
    CACHE.clear()

    return f"✅ **System Cache Cleared**\n\nCleared {cache_stats} cached entries.\n\n**Impact:**\n- All market data will be re-fetched\n- Sentiment analysis will refresh from live sources\n- Economic calendar will reload\n- Portfolio allocations will recalculate\n\n**Note:** Next analysis may take longer as data is refreshed."

def create_new_portfolio_visualization(allocation_text: str, budget: float, risk_level: str):
    """Create comprehensive portfolio visualization with risk overlay"""
    try:
        # Parse allocation data from text
        lines = allocation_text.split('\n')
        tickers, weights, amounts = [], [], []

        for line in lines:
            if '|' in line and any(ticker in line for ticker in TICKERS):
                parts = [p.strip() for p in line.split('|')]
                if len(parts) >= 4:
                    try:
                        ticker = parts[1]
                        weight = float(parts[2].replace('%', ''))
                        amount = float(parts[3].replace('$', '').replace(',', ''))

                        tickers.append(ticker)
                        weights.append(weight)
                        amounts.append(amount)
                    except:
                        continue

        if not tickers:
            return None

        # Create 2x2 subplot layout
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                f"Portfolio Allocation - {risk_level.title()} Risk",
                "Investment Distribution ($)",
                "Risk-Return Analysis",
                "Sector Diversification"
            ),
            specs=[
                [{"type": "pie"}, {"type": "bar"}],
                [{"type": "scatter"}, {"type": "pie"}]
            ]
        )

        #Color scheme
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#FFB347', '#98FB98']

        # 1. Main allocation pie chart with styling
        fig.add_trace(
            go.Pie(
                labels=[f"{t}\n({TICKER_NAMES.get(t, t)})" for t in tickers],
                values=weights,
                hole=0.4,
                marker=dict(
                    colors=colors[:len(tickers)],
                    line=dict(color='#000000', width=2)
                ),
                textinfo='label+percent',
                textposition='auto',
                textfont=dict(size=12),
                name="Portfolio %"
            ),
            row=1, col=1
        )

        # 2. Investment amounts bar chart
        fig.add_trace(
            go.Bar(
                x=tickers,
                y=amounts,
                text=[f'${amt:,.0f}' for amt in amounts],
                textposition='auto',
                marker=dict(
                    color=amounts,
                    colorscale='Viridis',
                    showscale=False,
                    line=dict(color='rgba(255,255,255,0.8)', width=1)
                ),
                name="Investment Amount"
            ),
            row=1, col=2
        )

        # 3. Risk-return scatter plot
        risk_data, return_data, sharpe_data = [], [], []
        for ticker in tickers:
            market_data = CACHE.get('market_data', {}).get(ticker, {})
            if market_data:
                risk_data.append(market_data.get('volatility', 0.15) * 100)
                return_data.append(market_data.get('annual_return', 0.08) * 100)
                sharpe_data.append(market_data.get('sharpe', 0.5))
            else:
                risk_data.append(15)
                return_data.append(8)
                sharpe_data.append(0.5)

        # Create scatter with styling
        fig.add_trace(
            go.Scatter(
                x=risk_data,
                y=return_data,
                mode='markers+text',
                text=tickers,
                textposition='top center',
                marker=dict(
                    size=[w*2.5 for w in weights],  # Size by portfolio weight
                    color=sharpe_data,
                    colorscale='RdYlGn',
                    showscale=True,
                    colorbar=dict(title="Sharpe Ratio", x=1.02),
                    sizemode='area',
                    sizeref=2.*max(weights)/(40.**2),
                    sizemin=8,
                    line=dict(width=2, color='rgba(255,255,255,0.8)')
                ),
                hovertemplate="<b>%{text}</b><br>Risk: %{x:.1f}%<br>Return: %{y:.1f}%<br>Weight: %{marker.size}%<extra></extra>",
                name="Risk-Return Profile"
            ),
            row=2, col=1
        )

        # 4. Sector allocation pie chart
        sector_data = {}
        for i, ticker in enumerate(tickers):
            sector = get_stock_sector(ticker)
            if sector in sector_data:
                sector_data[sector] += weights[i]
            else:
                sector_data[sector] = weights[i]

        if sector_data:
            fig.add_trace(
                go.Pie(
                    labels=list(sector_data.keys()),
                    values=list(sector_data.values()),
                    hole=0.3,
                    marker=dict(
                        colors=colors[len(tickers):],
                        line=dict(color='#000000', width=1)
                    ),
                    textinfo='label+percent',
                    textfont=dict(size=11),
                    name="Sector Allocation"
                ),
                row=2, col=2
            )

        # Update layout with  dark theme
        fig.update_layout(
            title=dict(
                text=f"RebalanceAI Portfolio Dashboard - ${budget:,.0f} Budget",
                font=dict(size=20, color='white'),
                x=0.5
            ),
            showlegend=False,
            height=900,
            template="plotly_dark",
            paper_bgcolor='rgba(15,15,35,0.9)',
            plot_bgcolor='rgba(15,15,35,0.9)',
            font=dict(color='white', family="Inter, Arial, sans-serif"),
            margin=dict(l=50, r=50, t=80, b=50)
        )

        # Update subplot titles with styling
        fig.update_annotations(font=dict(color='white', size=14, family="Inter"))

        # Update axes with better styling
        fig.update_xaxes(title_text="Stock", row=1, col=2, color='white', gridcolor='rgba(255,255,255,0.1)')
        fig.update_yaxes(title_text="Amount ($)", row=1, col=2, color='white', gridcolor='rgba(255,255,255,0.1)')
        fig.update_xaxes(title_text="Risk (Volatility %)", row=2, col=1, color='white', gridcolor='rgba(255,255,255,0.1)')
        fig.update_yaxes(title_text="Return (%)", row=2, col=1, color='white', gridcolor='rgba(255,255,255,0.1)')

        return fig

    except Exception as e:
        print(f"Visualization error: {e}")
        return None

# CSS for professional appearance
new_css = """
/* Professional RebalanceAI Theme */
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

.gradio-container {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif !important;
    background: linear-gradient(135deg, #0f0f23 0%, #1a1a2e 100%);
    color: #e2e8f0;
    min-height: 100vh;
}

/* Header with animation */
.main-header {
    background: linear-gradient(135deg, #6B46C1 0%, #9333EA 50%, #A855F7 100%);
    padding: 2.5rem;
    border-radius: 20px;
    margin-bottom: 2rem;
    box-shadow: 0 20px 60px rgba(147, 51, 234, 0.4);
    position: relative;
    overflow: hidden;
}

.main-header::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 200%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.1), transparent);
    animation: shimmer 4s infinite;
}

@keyframes shimmer {
    0% { left: -100%; }
    100% { left: 100%; }
}

.main-header h1 {
    color: white !important;
    font-size: 2.5rem !important;
    font-weight: 800 !important;
    margin-bottom: 0.5rem !important;
    text-shadow: 0 4px 8px rgba(0,0,0,0.3);
    letter-spacing: -0.5px;
}

.main-header p {
    color: rgba(255, 255, 255, 0.95) !important;
    font-size: 1.2rem !important;
    font-weight: 500 !important;
    margin-bottom: 1rem !important;
}

.feature-badge {
    display: inline-block;
    background: rgba(255, 255, 255, 0.2);
    border: 1px solid rgba(255, 255, 255, 0.3);
    padding: 0.5rem 1rem;
    border-radius: 25px;
    margin: 0.3rem;
    font-size: 0.9rem;
    font-weight: 600;
    backdrop-filter: blur(10px);
    transition: all 0.3s ease;
}

.feature-badge:hover {
    background: rgba(255, 255, 255, 0.3);
    transform: translateY(-2px);
}

/* Button styling */
.gr-button-primary {
    background: linear-gradient(135deg, #7C3AED, #9333EA) !important;
    border: none !important;
    color: white !important;
    font-weight: 600 !important;
    font-size: 1rem !important;
    padding: 1rem 2rem !important;
    border-radius: 12px !important;
    box-shadow: 0 6px 20px rgba(147, 51, 234, 0.4) !important;
    transition: all 0.3s ease !important;
    text-transform: none !important;
}

.gr-button-primary:hover {
    transform: translateY(-3px);
    box-shadow: 0 8px 30px rgba(147, 51, 234, 0.6) !important;
    background: linear-gradient(135deg, #8B5CF6, #A855F7) !important;
}

.gr-button-secondary {
    background: rgba(30, 30, 46, 0.9) !important;
    border: 2px solid rgba(147, 51, 234, 0.4) !important;
    color: #e2e8f0 !important;
    font-weight: 500 !important;
    border-radius: 12px !important;
    transition: all 0.3s ease !important;
}

.gr-button-secondary:hover {
    border-color: #9333EA !important;
    background: rgba(147, 51, 234, 0.1) !important;
    transform: translateY(-2px);
}

/* Form elements */
.gr-textbox, .gr-number, .gr-dropdown {
    font-family: 'Inter', sans-serif !important;
    background: rgba(15, 15, 35, 0.8) !important;
    border: 2px solid rgba(147, 51, 234, 0.3) !important;
    border-radius: 12px !important;
    color: #e2e8f0 !important;
    font-size: 1rem !important;
    transition: all 0.3s ease !important;
}

.gr-textbox:focus, .gr-number:focus, .gr-dropdown:focus {
    border-color: #9333EA !important;
    box-shadow: 0 0 0 4px rgba(147, 51, 234, 0.2) !important;
    background: rgba(15, 15, 35, 0.95) !important;
}

/* Radio buttons */
.gr-radio label {
    background: rgba(15, 15, 35, 0.8) !important;
    border: 2px solid rgba(147, 51, 234, 0.3) !important;
    border-radius: 12px !important;
    padding: 1rem 1.5rem !important;
    color: #cbd5e1 !important;
    font-weight: 500 !important;
    font-size: 0.95rem !important;
    transition: all 0.3s ease !important;
    cursor: pointer !important;
    margin: 0.5rem !important;
}

.gr-radio label:hover {
    border-color: #9333EA !important;
    background: rgba(147, 51, 234, 0.1) !important;
    transform: translateY(-1px);
}

.gr-radio input[type="radio"]:checked + label {
    background: linear-gradient(135deg, #7C3AED, #9333EA) !important;
    color: white !important;
    border-color: #9333EA !important;
    box-shadow: 0 4px 15px rgba(147, 51, 234, 0.4) !important;
    font-weight: 600 !important;
}

/* Tabs */
.tab-nav {
    font-weight: 600 !important;
    font-size: 1rem !important;
    padding: 1rem 2rem !important;
    border-radius: 12px !important;
    transition: all 0.3s ease !important;
    border: 2px solid transparent !important;
}

.tab-nav:hover {
    background: rgba(147, 51, 234, 0.2) !important;
    border-color: rgba(147, 51, 234, 0.3) !important;
}

.tab-nav.selected {
    background: linear-gradient(135deg, #7C3AED, #9333EA) !important;
    color: white !important;
    border: none !important;
    box-shadow: 0 4px 15px rgba(147, 51, 234, 0.4) !important;
}

/* Markdown styling */
.gr-markdown {
    font-family: 'Inter', sans-serif !important;
    line-height: 1.8 !important;
    color: #e2e8f0 !important;
}

.gr-markdown h1, .gr-markdown h2, .gr-markdown h3 {
    font-weight: 700 !important;
    color: #f1f5f9 !important;
    margin-bottom: 1rem !important;
}

.gr-markdown h2 {
    font-size: 1.8rem !important;
    color: #A855F7 !important;
    border-bottom: 2px solid rgba(147, 51, 234, 0.3);
    padding-bottom: 0.5rem;
}

.gr-markdown h3 {
    font-size: 1.4rem !important;
    color: #C084FC !important;
}

.gr-markdown table {
    background: rgba(15, 15, 35, 0.6) !important;
    border-radius: 12px !important;
    overflow: hidden !important;
    border: 1px solid rgba(147, 51, 234, 0.2) !important;
}

.gr-markdown th {
    background: rgba(147, 51, 234, 0.3) !important;
    color: white !important;
    font-weight: 600 !important;
    padding: 1rem !important;
}

.gr-markdown td {
    padding: 0.8rem !important;
    border-bottom: 1px solid rgba(147, 51, 234, 0.1) !important;
}

/* Progress bars */
.gr-progress-bar {
    background: linear-gradient(135deg, #7C3AED, #9333EA) !important;
    border-radius: 8px !important;
}

/* Examples */
.gr-examples {
    background: rgba(15, 15, 35, 0.6) !important;
    border-radius: 16px !important;
    border: 1px solid rgba(147, 51, 234, 0.2) !important;
    padding: 1.5rem !important;
}

.gr-example {
    background: rgba(30, 30, 46, 0.8) !important;
    border: 1px solid rgba(147, 51, 234, 0.3) !important;
    color: #cbd5e1 !important;
    border-radius: 8px !important;
    transition: all 0.3s ease !important;
}

.gr-example:hover {
    background: rgba(147, 51, 234, 0.2) !important;
    border-color: #9333EA !important;
    transform: translateY(-1px);
}

/* Responsive design */
@media (max-width: 768px) {
    .main-header h1 {
        font-size: 2rem !important;
    }

    .main-header p {
        font-size: 1rem !important;
    }

    .gr-button-primary {
        padding: 0.8rem 1.5rem !important;
        font-size: 0.9rem !important;
    }
}
"""

# Create the complete Gradio interface
with gr.Blocks(
    theme=gr.themes.Base(primary_hue="purple", secondary_hue="slate"),
    title="RebalanceAI - Complete AI Investment Co-Pilot",
    css=new_css
) as demo:

    # Header
    gr.HTML("""
    <div class="main-header">
        <h1>🚀 RebalanceAI - Complete AI Investment Co-Pilot</h1>
        <p><strong>Investment analysis with real-time data integration</strong></p>
        <p>Combining quantitative finance, sentiment analysis, economic calendar, and AI reasoning</p>
        <div style="margin-top: 1.5rem;">
            <span class="feature-badge">📊 Live Market Data</span>
            <span class="feature-badge">📰 News Sentiment</span>
            <span class="feature-badge">📱 Reddit Analysis</span>
            <span class="feature-badge">📅 Economic Calendar</span>
            <span class="feature-badge">🤖 ReAct Agent</span>
            <span class="feature-badge">🔬 Multi-LLM</span>
        </div>
    </div>
    """)

    with gr.Tabs() as main_tabs:
        # Tab 1: About RebalanceAI
        with gr.Tab("📖 About", elem_id="about-tab"):
            gr.Markdown("""
            ## 🚀 RebalanceAI - AI-Powered Investment Co-Pilot

            ### Group 2 - Shreyas Sreenivas, Anagha Veena Sanjeev, Harshitha Chandrashekar

            RebalanceAI combines AI reasoning with real-time financial data to provide institutional-quality investment analysis. Built with LangChain's ReAct agent, it orchestrates multiple data sources for transparent, actionable investment recommendations.

            ### 🎯 What Each Tab Does

            #### Main AI Investment Co-Pilot

            **🤖 AI Investment Co-Pilot** - Ask complex investment questions to the ReAct agent for comprehensive analysis combining technical, fundamental, sentiment, and economic data

            #### Individual Tools for you to explore

            **💼 Smart Portfolio** - Create optimized portfolios with budget allocation, risk profiles (Conservative/Moderate/Aggressive), and economic calendar integration

            **📊 Stock Comparison** - Head-to-head analysis of multiple stocks across performance, technical indicators, fundamentals, and sentiment with winner determination

            **📅 Economic Calendar** - Analyze current economic conditions using FRED data, interest rates, economic indicators, and stock-specific impact assessment

            **🔬 Model Comparison** - Test and compare different AI models (OpenAI vs Google Gemini) for investment analysis performance and quality

            ### ⚙️ Core Technology

            **Data Sources**: Yahoo Finance, NewsAPI, Reddit, Federal Reserve (FRED)
            **Analysis**: Technical (RSI, MACD) + Fundamental (P/E, growth) + Sentiment (news + social) + Economic (Fed data)
            **AI Framework**: ReAct agent with intelligent tool orchestration and transparent reasoning
            """)

            gr.Markdown("---")
            gr.Markdown("### 🔧 System Status & Management")

            with gr.Row():
                refresh_status_btn = gr.Button("🔄 Refresh System Status", variant="primary", scale=2)
                clear_all_cache_btn = gr.Button("🗑️ Clear All Cache", variant="secondary", scale=1)

            # System status in row layout (5 columns now)
            with gr.Row():
                with gr.Column(scale=1):
                    api_status_output = gr.Markdown(
                        value=get_api_status(),
                        label="API Status"
                    )
                with gr.Column(scale=1):
                    cache_status_output = gr.Markdown(
                        value=get_cache_status(),
                        label="Cache Status"
                    )
                with gr.Column(scale=1):
                    tools_status_output = gr.Markdown(
                        value=get_tools_status(),
                        label="Tools Status"
                    )
                    gr.Markdown("#### multi_llm_comparison_tool is an Independant addition in the app to demonstrate multi-LLM comparison")
                with gr.Column(scale=1):
                    performance_output = gr.Markdown(
                        value=get_performance_info(),
                        label="Performance Info"
                    )
                with gr.Column(scale=1):
                    secrets_status_output = gr.Markdown(
                        value=get_secrets_status(),
                        label="Environment Info"
                    )

            # Update button click handlers
            def refresh_all_status():
                return (
                    get_api_status(),
                    get_cache_status(),
                    get_tools_status(),
                    get_performance_info(),
                    get_secrets_status()
                )

            def clear_cache_and_update():
                global CACHE
                cache_stats = len(CACHE)
                CACHE.clear()

                # Return updated status for all columns
                return (
                    get_api_status(),
                    get_cache_status(),
                    get_tools_status(),
                    get_performance_info(),
                    get_secrets_status()
                )

            refresh_status_btn.click(
                fn=refresh_all_status,
                outputs=[api_status_output, cache_status_output, tools_status_output, performance_output, secrets_status_output]
            )

            clear_all_cache_btn.click(
                fn=clear_cache_and_update,
                outputs=[api_status_output, cache_status_output, tools_status_output, performance_output, secrets_status_output]
            )

            # Keep only the essential disclaimer
            gr.HTML("""
            <div style="background: rgba(15, 15, 35, 0.9); border-radius: 16px; padding: 2rem; border: 1px solid rgba(220, 38, 127, 0.3); margin-top: 2rem;">
                <h3 style="color: #F472B6; margin-bottom: 1rem;">⚠️ Important Disclaimers</h3>
                <div style="color: #fbbf24; line-height: 1.8;">
                    <p><strong>Educational Purpose Only:</strong> RebalanceAI is designed for educational and research purposes. All analysis and recommendations are for informational use only.</p>
                    <p><strong>Not Financial Advice:</strong> This system does not provide personalized financial advice. Always consult with qualified financial advisors before making investment decisions.</p>
                    <p><strong>Risk Consideration:</strong> All investments involve risk of loss. Past performance does not guarantee future results. Consider your risk tolerance and financial situation.</p>
                </div>
            </div>
            """)

        # Tab 2: AI Investment Advisor (ReAct Agent)
        with gr.Tab("🤖 AI Investment Co-Pilot", elem_id="advisor-tab"):
            gr.Markdown("### Ask Investment Questions - Powered by ReAct Agent with Tool Orchestration")

            advisor_query_input = gr.Textbox(
                label="💭 Your Investment Question",
                placeholder="Examples:\n• Should I invest in Apple considering economic events and sentiment?\n• Compare Tesla vs Nvidia across all metrics for long-term growth\n• What's the best tech stock portfolio for moderate risk tolerance?\n• Analyze Microsoft's investment potential with economic calendar impact",
                lines=5,
                info="Ask any investment question for comprehensive ReAct agent analysis"
            )

            ask_advisor_btn = gr.Button("🔍 Get AI Co-Pilot Analysis", variant="primary", size="lg")

            advisor_output = gr.Markdown(
                value="*Your comprehensive ReAct agent investment analysis will appear here*",
                label="AI Co-Pilot Analysis Results"
            )

            ask_advisor_btn.click(
                fn=process_ai_advisor_query,
                inputs=advisor_query_input,
                outputs=advisor_output,
                show_progress=True
            )

            gr.Examples(
                examples=[
                    "Should I invest $50,000 in Apple stock considering current sentiment and economic events?",
                    "Compare Tesla vs Nvidia vs Microsoft for long-term investment potential",
                    "What are the best tech stocks with positive sentiment and low economic risk?",
                    "Create a $25,000 moderate risk portfolio focused on growth stocks",
                    "Analyze Google's investment prospects considering all available data"
                ],
                inputs=advisor_query_input,
                label="📌 Try These Investment Questions"
            )

        # Tab 3: Smart Portfolio Allocation
        with gr.Tab("💼 Smart Portfolio", elem_id="portfolio-tab"):
            gr.Markdown("### Create AI-Optimized Portfolios with Economic Calendar Integration")

            with gr.Row():
                with gr.Column(scale=1):
                    budget_input = gr.Number(
                        label="💵 Investment Budget ($)",
                        value=25000,
                        minimum=1000,
                        maximum=10000000,
                        info="Your total investment amount"
                    )
                with gr.Column(scale=1):
                    risk_input = gr.Radio(
                        ["conservative", "moderate", "aggressive"],
                        label="⚖️ Risk Profile",
                        value="moderate",
                        info="Risk tolerance affects economic event sensitivity"
                    )
                with gr.Column(scale=1):
                    focus_input = gr.Radio(
                        ["Tech Heavy", "Diversified", "S&P Top 10", "Financial Focus", "Healthcare Focus"],
                        label="🎯 Investment Focus",
                        value="Tech Heavy",
                        info="Sector allocation preference"
                    )

            with gr.Row():
                create_portfolio_btn = gr.Button("🚀 Generate Smart Portfolio", variant="primary", size="lg", scale=3)
                clear_cache_btn = gr.Button("🗑️ Clear Cache", variant="secondary", scale=1)

            portfolio_output = gr.Markdown(
                value="*Your AI-optimized portfolio with economic calendar integration will appear here*",
                label="Portfolio Analysis Results"
            )

            portfolio_viz = gr.Plot(
                label="Interactive Portfolio Dashboard",
                visible=False
            )

            # Wire up functionality
            create_portfolio_btn.click(
                fn=process_portfolio_request,
                inputs=[budget_input, risk_input, focus_input],
                outputs=[portfolio_output, portfolio_viz],
                show_progress=True
            )

            clear_cache_btn.click(
                fn=clear_all_system_cache,
                outputs=portfolio_output
            )

            gr.Examples(
                examples=[
                    [100000, "conservative", "Diversified"],
                    [50000, "moderate", "Tech Heavy"],
                    [25000, "aggressive", "S&P Top 10"],
                    [75000, "moderate", "Financial Focus"]
                ],
                inputs=[budget_input, risk_input, focus_input],
                label="📌 Try These Portfolio Configurations"
            )

        # Tab 4: Stock Comparison Engine
        with gr.Tab("📊 Stock Comparison", elem_id="comparison-tab"):
            gr.Markdown("### Head-to-Head Stock Analysis Across All Metrics")

            comparison_tickers_input = gr.Textbox(
                label="🏢 Stocks to Compare",
                placeholder="AAPL,MSFT,GOOGL,NVDA",
                info="Enter 2-6 stock tickers separated by commas for comprehensive comparison"
            )

            compare_stocks_btn = gr.Button("⚔️ Compare Stocks", variant="primary", size="lg")

            comparison_output = gr.Markdown(
                value="*Comprehensive stock comparison results will appear here*",
                label="Stock Comparison Results"
            )

            compare_stocks_btn.click(
                fn=process_stock_comparison,
                inputs=comparison_tickers_input,
                outputs=comparison_output,
                show_progress=True
            )

            gr.Examples(
                examples=[
                    "AAPL,MSFT,GOOGL",
                    "TSLA,NVDA,AMD",
                    "JPM,V,MA,BRK-B",
                    "JNJ,UNH,LLY,ABBV",
                    "AMZN,META,NFLX"
                ],
                inputs=comparison_tickers_input,
                label="📌 Popular Stock Comparisons"
            )

        # Tab 5: Economic Calendar Analysis
        with gr.Tab("📅 Economic Calendar", elem_id="calendar-tab"):
            gr.Markdown("### Economic Events Impact Analysis & Timing Intelligence")

            with gr.Row():
                with gr.Column(scale=2):
                    econ_tickers_input = gr.Textbox(
                        label="🏢 Stocks to Analyze (optional)",
                        placeholder="AAPL,MSFT,GOOGL or leave empty for general calendar",
                        info="Enter specific tickers for targeted economic impact analysis"
                    )
                with gr.Column(scale=1):
                    econ_days_input = gr.Number(
                        label="📅 Days Ahead",
                        value=45,
                        minimum=7,
                        maximum=120,
                        info="Economic event analysis horizon"
                    )

            analyze_economic_btn = gr.Button("📊 Analyze Economic Impact", variant="primary", size="lg")

            economic_output = gr.Markdown(
                value="*Economic calendar analysis with stock-specific impact assessment will appear here*",
                label="Economic Calendar Analysis Results"
            )

            analyze_economic_btn.click(
                fn=process_economic_calendar_analysis,
                inputs=[econ_tickers_input, econ_days_input],
                outputs=economic_output,
                show_progress=True
            )

            gr.Examples(
                examples=[
                    ["AAPL,MSFT,GOOGL", 30],
                    ["TSLA,NVDA", 45],
                    ["JPM,V,MA", 60],
                    ["", 30],
                    ["JNJ,UNH,LLY", 90]
                ],
                inputs=[econ_tickers_input, econ_days_input],
                label="📌 Economic Impact Analysis Examples"
            )

        # Tab 6: Multi-LLM Model Comparison
        with gr.Tab("🔬 Model Comparison", elem_id="models-tab"):
            gr.Markdown("### Compare AI Models for Investment Analysis Performance")

            model_test_query_input = gr.Textbox(
                label="🧪 Test Query for Model Comparison",
                value="Analyze Apple stock for long-term investment potential considering current market conditions, sentiment, and economic outlook",
                lines=4,
                info="Investment analysis query to test across different AI models"
            )

            compare_models_btn = gr.Button("🔬 Compare AI Models", variant="primary", size="lg")

            models_output = gr.Markdown(
                value="*Multi-LLM model comparison results with performance metrics will appear here*",
                label="Model Comparison Results"
            )

            compare_models_btn.click(
                fn=process_model_comparison,
                inputs=model_test_query_input,
                outputs=models_output,
                show_progress=True
            )

            gr.Examples(
                examples=[
                    "Analyze Tesla stock for growth investment potential",
                    "Compare Microsoft vs Google for conservative investors",
                    "What are the best AI-related investment opportunities in 2024?",
                    "Evaluate Nvidia's long-term prospects in semiconductor market",
                    "Should I invest in healthcare stocks given current market conditions?"
                ],
                inputs=model_test_query_input,
                label="📌 Model Testing Queries"
            )

    # Footer with comprehensive disclaimer
    gr.Markdown("""
    ---
    ### ⚠️ Important Investment Disclaimer

    **Educational and Research Tool Only**: RebalanceAI is designed exclusively for educational and research purposes. This system provides **analytical insights, not financial advice**.

    **Critical Reminders**:
    - 📊 **Past Performance**: Historical data does not guarantee future investment results
    - 🔍 **Due Diligence**: Always conduct thorough independent research before investing
    - 💼 **Professional Consultation**: Consult qualified financial advisors for investment decisions
    - ⚖️ **Risk Assessment**: Consider your personal financial situation and risk tolerance
    - 🌐 **Market Volatility**: Financial markets involve inherent risks and unpredictability
    - 📈 **Diversification**: No investment strategy guarantees profits or prevents losses

    **Data Sources**: Yahoo Finance (market data), NewsAPI (financial news), Reddit (social sentiment), Economic calendar APIs

    **AI Limitations**: AI analysis is based on available data and algorithms. Human judgment and professional expertise remain essential for investment decisions.

    ---

    **RebalanceAI v2.0** | Built with LangChain, OpenAI, and financial analysis algorithms | For educational use only
    """)

print("✅ Complete RebalanceAI Gradio Interface Ready!")
print("🎯 Features: Portfolio optimization, ReAct agent, stock comparison, economic calendar, multi-LLM")
print("🚀 Launch command: demo.launch(share=True)")

✅ Complete RebalanceAI Gradio Interface Ready!
🎯 Features: Portfolio optimization, ReAct agent, stock comparison, economic calendar, multi-LLM
🚀 Launch command: demo.launch(share=True)


## 🚀 Launch RebalanceAI - Complete System

### Final System Launch

Your complete RebalanceAI system is now ready! Here's how to launch and use it:

**Launch Command**: `demo.launch(share=True)`

### Complete Feature Set:

1. **💼 Smart Portfolio Allocation**: Economic calendar-integrated portfolio optimization
2. **🤖 AI Investment Co-Pilot**: ReAct agent with comprehensive analysis capabilities  
3. **📊 Stock Comparison**: Head-to-head analysis across all metrics
4. **📅 Economic Calendar**: Event impact analysis and timing intelligence
5. **🔬 Multi-LLM Comparison**: AI model performance testing
6. **🔧 System Management**: Status monitoring and cache management

### What Makes This Special:

- **Real-time Data Integration**: Live market data, news, social media sentiment
- **Economic Event Awareness**: Position sizing based on upcoming events
- **Multi-Factor Analysis**: Technical + Fundamental + Sentiment + Economic
- **Professional Visualizations**: Interactive portfolio dashboards
- **Transparent Reasoning**: See exactly how AI makes investment decisions

In [45]:
# ===================================================================
# Final Launch Instructions & System Summary
# ===================================================================

print("="*70)
print("🚀 REBALANCEAI COMPLETE SYSTEM READY FOR LAUNCH")
print("="*70)

print("\n📋 SYSTEM COMPONENTS LOADED:")
print("✅ Market Data Tool with comprehensive technical/fundamental analysis")
print("✅ Multi-Source Sentiment Analysis (NewsAPI + Reddit integration)")
print("✅ Economic Calendar Tool with event impact assessment")
print("✅ Portfolio Allocator with economic risk adjustments")
print("✅ Stock Comparison Engine with winner determination")
print("✅ ReAct Agent with intelligent tool orchestration")
print("✅ Multi-LLM Comparison System")

print(f"\n🛠️ TOOLS AVAILABLE TO REACT AGENT:")
for i, tool in enumerate(new_tools, 1):
    print(f"  {i}. {tool.name}: {tool.description}")

print(f"multi_llm_comparison_tool is an Independant addition in the app to demonstrate multi-LLM comparison")

print(f"\n🤖 AI AGENT CONFIGURATION:")
print(f"  • Model: GPT-4o-mini (OpenAI)")
print(f"  • Agent Type: ReAct (Reasoning + Acting)")
print(f"  • Max Iterations: 6 steps")
print(f"  • Memory: Conversation history enabled")
print(f"  • Error Handling: Graceful recovery")

print(f"\n📊 DATA SOURCES:")
print(f"  • Market Data: Yahoo Finance (real-time)")
print(f"  • News Sentiment: NewsAPI (7-day rolling)")
print(f"  • Social Sentiment: Reddit API (WSB, stocks, investing)")
print(f"  • Economic Events: Fed calendar + economic indicators")

print(f"\n🎯 UNIQUE FEATURES:")
print(f"  • Economic calendar integration in portfolio allocation")
print(f"  • Risk-profile-appropriate economic event sensitivity")
print(f"  • Multi-factor scoring with transparent weighting")
print(f"  • Intelligent caching for optimal performance")
print(f"  • Professional-grade visualizations")
print(f"  • Comprehensive error handling and user guidance")

print(f"\n💡 USAGE EXAMPLES:")
print(f"  Portfolio: 'Create a $50,000 moderate risk tech-heavy portfolio'")
print(f"  Analysis: 'Should I invest in Apple considering current conditions?'")
print(f"  Comparison: 'Compare Tesla vs Nvidia for long-term growth'")
print(f"  Economic: 'What events might impact my tech stocks this month?'")

# System health check
print(f"\n🔍 SYSTEM HEALTH CHECK:")
health_status = []

# Check APIs
try:
    if OPENAI_API_KEY and len(OPENAI_API_KEY) > 10:
        health_status.append("✅ OpenAI API: Configured")
    else:
        health_status.append("❌ OpenAI API: Missing")

    if NEWSAPI_KEY and len(NEWSAPI_KEY) > 10:
        health_status.append("✅ NewsAPI: Configured")
    else:
        health_status.append("❌ NewsAPI: Missing")

    if REDDIT_CLIENT_ID and len(REDDIT_CLIENT_ID) > 5:
        health_status.append("✅ Reddit API: Configured")
    else:
        health_status.append("❌ Reddit API: Missing")

    if GOOGLE_API_KEY and len(GOOGLE_API_KEY) > 10:
        health_status.append("✅ Google API: Configured")
    else:
        health_status.append("❌ Google API: Missing")

except Exception as e:
    health_status.append(f"⚠️ API Check Error: {str(e)}")

for status in health_status:
    print(f"  {status}")

# Check agent
try:
    test_response = llm.predict("Hello")
    print(f"  ✅ LLM Connection: Working")
except Exception as e:
    print(f"  ❌ LLM Connection: Error - {str(e)}")

print(f"  ✅ Agent Executor: Ready")
print(f"  ✅ Gradio Interface: Loaded")

# Cache status
print(f"\n💾 CACHE STATUS:")
print(f"  • Market Data: {len(CACHE.get('market_data', {}))} stocks cached")
print(f"  • Sentiment Data: {len(CACHE.get('sentiment', {}))} stocks cached")
print(f"  • Economic Calendar: {len(CACHE.get('economic_calendar', {}))} stocks cached")

print(f"\n🚀 LAUNCH INSTRUCTIONS:")
print("="*50)
print("Run this command to launch the complete RebalanceAI system:")
print()
print("    demo.launch(share=True)")
print()
print("This will:")
print("  • Start the Gradio web interface")
print("  • Generate a public share link")
print("  • Enable access from any device")
print("  • Provide full RebalanceAI functionality")

print(f"\n📱 INTERFACE FEATURES:")
print(f"  • Real-time progress indicators for all operations")
print(f"  • Interactive portfolio visualizations with risk overlays")
print(f"  • Comprehensive error handling with user guidance")
print(f"  • Example queries and configurations for easy testing")
print(f"  • Mobile-responsive design for access anywhere")

print(f"\n🏆 WHAT MAKES REBALANCEAI SPECIAL:")
print("="*50)
print("✨ First AI investment system with economic calendar integration")
print("✨ Multi-source sentiment analysis (professional + retail)")
print("✨ ReAct agent with transparent reasoning process")
print("✨ Risk-profile-aware economic event sensitivity")
print("✨ Complete transparency in decision-making process")

print("\n" + "="*70)
print("🎯 REBALANCEAI IS READY FOR USE!")
print("="*70)
print()
print("Launch with: demo.launch(share=True)")
print()

🚀 REBALANCEAI COMPLETE SYSTEM READY FOR LAUNCH

📋 SYSTEM COMPONENTS LOADED:
✅ Market Data Tool with comprehensive technical/fundamental analysis
✅ Multi-Source Sentiment Analysis (NewsAPI + Reddit integration)
✅ Economic Calendar Tool with event impact assessment
✅ Portfolio Allocator with economic risk adjustments
✅ Stock Comparison Engine with winner determination
✅ ReAct Agent with intelligent tool orchestration
✅ Multi-LLM Comparison System

🛠️ TOOLS AVAILABLE TO REACT AGENT:
  1. gather_market_data: Gather comprehensive market data including technical indicators, fundamentals, and performance metrics. Input: ticker symbols like AAPL or AAPL,MSFT,GOOGL
  2. analyze_sentiment: Analyze market sentiment combining news and Reddit sources. Input: ticker symbols like AAPL or AAPL,MSFT,GOOGL
  3. create_portfolio: Create optimized portfolio allocation with current economic conditions integration. Input format: budget=X,risk=conservative/moderate/aggressive
  4. analyze_economic_calendar: 

In [46]:
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c544ea32587f786a29.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Thought: Analyzing the query to determine the necessary tools for comparing TSLA, NVDA, and AMD.
Action: gather_market_data
Action Input: TSLA,NVDA,AMD📊 Processing TSLA (Tesla)...
📊 Processing NVDA (Nvidia)...
Market Data Analysis:
TSLA: Price=$439.31, Sharpe=1.30, PE=135.6, Growth=-11.8%, RSI=48.9
NVDA: Price=$183.22, Sharpe=0.79, PE=44.5, Growth=55.6%, RSI=51.5Continuing the analysis to gather sentiment data for TSLA, NVDA, and AMD.
Action: analyze_sentiment
Action Input: TSLA,NVDA,AMD💭 Analyzing sentiment for TSLA (Tesla)...
🔍 Searching Reddit for $TSLA mentions...
💭 Analyzing sentiment for NVDA (Nvidia)...
🔍 Searching Reddit for $NVDA mentions...
Multi-Source Sentiment Analysis:
TSLA: 🟡 Neutral ⚡ | Combined=+0.15 (News=+0.00, Reddit=+0.49) | Sources: 10 articles, 10 posts
NVDA: 🟡 Neutral ⚡ | Combined=+0.28 (News=+0.20, Reddit=+0.48) | Sources: 10 articles, 13 postsContinuing the analysis to compare the stocks TSLA, NVDA, and AMD across all me